In [1]:
import cx_Oracle
import pandas as pd
import sqlalchemy as sa
import getpass
import numpy as np
import math
# import pyodbc
import pandas.io.sql as psql
import decimal
import datetime
from datetime import date
import csv
import sys
import sqlite3
import subprocess
import codecs
from time import gmtime, strftime
from timeit import default_timer as timer
from sqlalchemy.types import NVARCHAR
from sqlalchemy.types import NUMERIC
from sqlalchemy.types import FLOAT
sys.path.insert(0,'/sasdata/ci_brazil/cust_fund/One_Pagers/padrao_op/sasprog/')
import defs
engine_OUT = sa.create_engine("oracle+cx_oracle://[AN_RT_WS36]:@exa_gbgpaprdd_an",echo=False)

In [2]:
#Getting the current date#
todays_date = strftime("%d/%m/%Y", gmtime())
print (todays_date)
# data_hoje = datetime.date.today()
# data_hoje

14/03/2018


In [3]:
def sqlcol(dtf):    
    dtypedict = {}
    for i,j in zip(dtf.columns, dtf.dtypes):
        if "object" in str(j):
            dtypedict.update({i: sa.types.NVARCHAR(length=255)})

        if "datetime" in str(j):
            dtypedict.update({i: sa.types.DateTime()})

        if "float" in str(j):
            dtypedict.update({i: sa.types.Float(precision=3, asdecimal=True)})

        if "int" in str(j):
            dtypedict.update({i: sa.types.INT()})

        if "int64" in str(j):
            dtypedict.update({i: sa.types.INT()})

    return dtypedict

<b><h1>IMPORTING VIA VAREJO FILES</b></h1>

In [4]:
# Input of Via Varejo´s files
BASE_TOLUNA = pd.read_excel("/nfs/science/casino_brazil/luizsalv/Via_Varejo/input/BASE_TOLUNA_N.xlsx", sheet_name=0)

#Input Atributes RANKS
RANKS = pd.read_excel("/nfs/science/casino_brazil/luizsalv/Via_Varejo/input/BASE_TOLUNA.xlsx", sheet_name=2)

#Input ISC info
ISC_VALOR_N= pd.read_excel("/nfs/science/casino_brazil/luizsalv/Via_Varejo/input/ISC_VIA_VAREJO_N.xlsx", sheet_name=0)
# ISC_VALOR_N['ISC']=ISC_VALOR_N['ISC'].apply(lambda x: x.replace(',','.'))
ISC_VALOR_N['RESPID'] = ISC_VALOR_N['respid']
del(ISC_VALOR_N['respid'])

#Input metodologia (vv)
# METODOLOGIA= pd.read_excel("/nfs/science/casino_brazil/luizsalv/Via_Varejo/input/metodologia_jan.xlsx", sheet_name=0)
# METODOLOGIA.columns = map(unicode.upper, METODOLOGIA.columns)

#Input cadastro info
ISC_GEN = pd.read_csv('/nfs/science/casino_brazil/luizsalv/Via_Varejo/input/ISC_GEN.csv',sep=';')
ISC_LC = pd.read_csv('/nfs/science/casino_brazil/luizsalv/Via_Varejo/input/ISC_LC.csv',sep=';')
ISC_GEN = [ISC_GEN,ISC_LC]
ISC_GEN = pd.concat(ISC_GEN)
ISC_GEN['DEPTO'] =(ISC_GEN['DEPTO']).astype('str')
ISC_GEN['SETOR'] =(ISC_GEN['SETOR']).astype('str')

#Input cadastro completo, priorizando móveis
# CADASTRO_COMPLETO = pd.read_csv('/nfs/science/casino_brazil/luizsalv/Via_Varejo/input/CADASTRO_COMPLETO_VV.csv',sep=';')
# CADASTRO_COMPLETO.columns = map(str.upper, CADASTRO_COMPLETO.columns)


print('Done Imports')

Done Imports


<b><h1>Treating words</b></h1>

In [5]:
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('ã','a'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('á','a'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('à','a'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('â','a'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('ä','a'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('ç','c'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('é','e'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('è','e'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('ê','e'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('í','i'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('ñ','n'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('ö','o'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('ü','u'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('Ì','I'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('£',''))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('¥',''))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('€',''))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('¡','i'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('¿',''))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('ÃO','AO'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('Ç','c'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('ç','c'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('Â','A'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('Ã','A'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('ó','o'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('ò','o'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('"',''))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('³',''))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('²',''))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('¹',''))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('É','E'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('È','E'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('ú','u'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('Ú','U'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('Ù','U'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('Á','Ad'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('À','A'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('Ò','O'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('Ó','O'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('ô','o'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('ão','ao'))
ISC_GEN['DEPTO']=ISC_GEN['DEPTO'].apply(lambda x: x.replace('õe','oe'))

ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('ã','a'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('á','a'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('à','a'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('â','a'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('ä','a'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('ç','c'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('é','e'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('ê','e'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('è','e'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('í','i'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('ñ','n'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('ö','o'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('ü','u'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('Ì','I'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('£',''))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('¥',''))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('€',''))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('¡','i'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('¿',''))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('ÃO','AO'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('Ç','c'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('ç','c'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('Â','A'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('Ã','A'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('ó','o'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('ò','o'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('"',''))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('³',''))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('²',''))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('¹',''))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('É','E'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('È','E'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('ú','u'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('Ú','U'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('Ù','U'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('Á','A'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('À','A'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('Ò','O'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('Ó','O'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('ô','o'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('ão','ao'))
ISC_GEN['SETOR']=ISC_GEN['SETOR'].apply(lambda x: x.replace('õe','oe'))


<b><h1>Reading historical ISC values</b></h1>

In [75]:
ISC_HIST="""SELECT * FROM AN_RT_WS36.VV_ISC_HIST"""
ISC_HIST = pd.read_sql(ISC_HIST, engine_OUT)
ISC_HIST.columns = map(unicode.upper, ISC_HIST.columns)
ISC_HIST['RESPID'] =(ISC_HIST['RESPID']).astype('int')
ISC_HIST

,ISC,RESPID
0,78.6970954221222,226103
1,100,226112
2,94.5026396155086,226108
3,100,226109
4,93.1770040930653,226106
5,59.104857553732,226115
6,97.7079337384144,226117
7,94.1228948311851,226118
8,0,226126
9,0,226114


<b><h1>Appending new ISC values to the historical</b></h1>

In [82]:
ISC_VALOR = ISC_VALOR_N.append(ISC_HIST)

#Removing Duplicates from VV base
ISC_VALOR = ISC_VALOR.drop_duplicates(subset=['RESPID'], keep='first')
ISC_VALOR['ISC'] = pd.to_numeric(ISC_VALOR['ISC']).astype(float)
ISC_VALOR

,ISC,RESPID
0,65.971432,86244
1,73.397471,86190
2,100.000000,85537
3,1.241904,85850
4,70.000000,85901
5,60.986506,86258
6,98.927197,85595
7,94.977511,86221
8,75.330509,86250
9,70.000000,85898


<b><h1>Creating an historical data for ISC values</b></h1>

In [83]:
dtype5={
'ISC' :    sa.NVARCHAR(length=255),
'RESPID' : sa.NVARCHAR(length=255)}

ISC_VALOR.to_sql('vv_isc_hist',engine_OUT,schema=None,if_exists='replace', chunksize=500, index=False,dtype=dtype5)
print('Done uploading isc hist')

Done uploading isc hist


<b><h1>Reading the historical information from Exadata</b></h1>

In [57]:
CADASTRO_HIST="""
SELECT DISTINCT * FROM AN_RT_WS36.VV_CADASTRO"""
CADASTRO_HIST = pd.read_sql(CADASTRO_HIST, engine_OUT)
CADASTRO_HIST.columns = map(unicode.upper, CADASTRO_HIST.columns)
CADASTRO_HIST

,BANDEIRA_NUM_PEDIDO,DS_TIPO_ENTREGA,DATE_ID,DEPTO,METODOLOGIA,SETOR,UF,DT_NASC,SEXO
0,137831648CB,Estoque ViaVarejo,25/01/2018,Utilidades Dom?sticas,Data Entrega,Faqueiros,SP,22/06/1970,F
1,136925951CB,Estoque ViaVarejo,16/01/2018,Eletroport?teis,Data Entrega,M?quinas de Costura,MG,02/09/1958,F
2,137481801EX,Estoque Cnova,18/01/2018,Eletroport?teis,Data Entrega,Aspirador de P?,SP,11/07/1981,F
3,131340362PF,Estoque Cnova,17/01/2018,Eletroport?teis,Data Entrega,Cozinha Criativa,SP,06/09/1983,F
4,137052969EX,Estoque ViaVarejo,14/01/2018,Telefones e Celulares,Data Entrega,Smartphones Desbloqueados,BA,06/04/1972,M
5,128319838EX,Normal,16/01/2018,Automotivo,Data Prometida,Marketplace,SP,23/02/1980,F
6,131885295CB,Estoque ViaVarejo,16/01/2018,Eletroport?teis,Data Entrega,Cafeteira Capsula/Expresso,SP,05/05/1963,F
7,136506866CB,Normal,16/01/2018,Rel?gios,Data Prometida,Marketplace,RJ,23/09/1977,M
8,133173354EX,Normal,18/01/2018,Eletrodom?sticos,Data Prometida,Refrigeradores,MG,11/08/1953,F
9,136246371PF,Normal,19/01/2018,Beleza e Sa?de,Data Prometida,NAO_PRODUTO,PR,21/01/1980,F


<b><h1>Appending new info to Hist</b></h1>

In [16]:
ISC_GEN['BANDEIRA_NUM_PEDIDO'] = ISC_GEN['ID_COMPRA'].map(str) + ISC_GEN['BANDEIRA']
ISC_ALL = ISC_GEN.append(CADASTRO_HIST)
#Removing Duplicates from VV base
ISC_ALL = ISC_ALL.drop_duplicates(subset=['BANDEIRA_NUM_PEDIDO'], keep='first')
del(ISC_ALL['QTD_PRODUTOS'])
del(ISC_ALL['VALOR_PEDIDO'])
del(ISC_ALL['NR_ENTREGA'])
del(ISC_ALL['NR_LOJISTA'])
ISC_ALL

,BANDEIRA,BANDEIRA_NUM_PEDIDO,DATA_PONTO,DEPTO,DS_TIPO_ENTREGA,DT_NASC,DT_PEDIDO,ENT,ID_CLIENTE,ID_COMPRA,LOJISTA,NOTA,PONTO_CONTROLE,SEGMENTO,SETOR,SEXO,TIPO_LISTA,UF
0,CB,109884686CB,12/12/2016 14:14,Games,Normal,28/08/1964,25/11/2016,CLIENTE,7871999.0,109884686,FOX EXPRESS,0,NFS,MKTPLACE,Console,F,NaN,AL
1,EX,121523246EX,03/07/2017 12:33,Eletroportateis,Normal,24/04/1991,30/06/2017,CLIENTE,14879969.0,121523246,H2O Purificadores,0,NFS,MKTPLACE,NAO_PRODUTO,M,NaN,SP
2,PF,122100184PF,28/07/2017 21:28,Telefones e Celulares,Normal,01/07/1966,07/07/2017,CLIENTE,4112350.0,122100184,HM Eletro,0,IVE,MKTPLACE,NAO_PRODUTO,M,NaN,SP
6,PF,122362490PF,29/07/2017 01:38,Games,Normal,25/03/1978,12/07/2017,CLIENTE,113874.0,122362490,Fenix Shop,9,NFS,MKTPLACE,Marketplace,M,NaN,SP
7,PF,123098544PF,29/07/2017 06:47,Telefones e Celulares,Normal,06/08/1962,20/07/2017,CLIENTE,6718818.0,123098544,Lu Celulares,0,NFS,MKTPLACE,Smartphones Desbloqueados,M,NaN,SP
8,EX,126593904EX,13/09/2017 09:57,Pet Shop,Normal,02/07/1960,08/09/2017,CLIENTE,528983.0,126593904,Petvirtual,10,NFS,MKTPLACE,NAO_PRODUTO,F,NaN,SP
9,PF,127198020PF,25/10/2017 07:49,TVs e Acessorios,Normal,22/03/1985,20/09/2017,CLIENTE,13386086.0,127198020,VGSHOP,10,NFS,MKTPLACE,Marketplace,F,NaN,PE
10,EX,127579889EX,26/09/2017 23:58,Beleza e Saude,Normal,09/10/1978,24/09/2017,CLIENTE,7118062.0,127579889,Girafa,0,NFS,MKTPLACE,Marketplace,F,NaN,RJ
11,CB,127746636CB,29/09/2017 18:43,Telefones e Celulares,Outros,08/01/1988,27/09/2017,CLIENTE,15276594.0,127746636,Directstock,1,NFS,MKTPLACE,Smartphones Desbloqueados,M,NaN,SP
12,PF,127765204PF,01/11/2017 15:07,Moveis,Normal,04/09/1974,27/09/2017,CLIENTE,661744.0,127765204,Mezzanine,0,NFS,MKTPLACE,Marketplace,F,NaN,PR


<b><h1>Uploading cadastro to Exadata</b></h1>

In [25]:
ISC_ALL['NOTA'] =(ISC_ALL['NOTA']).astype('float')
ISC_ALL['ID_COMPRA'] =(ISC_ALL['ID_COMPRA']).astype('int')

dtype1={
'PONTO_CONTROLE' : sa.NVARCHAR(length=255) ,
'ENT' : sa.NVARCHAR(length=255) ,
'NR_ENTREGA' : sa.INTEGER ,
'TIPO_LISTA' : sa.NVARCHAR(length=255) ,
'BANDEIRA' : sa.NVARCHAR(length=255) ,
'BANDEIRA_NUM_PEDIDO' : sa.NVARCHAR(length=255) ,
'SEGMENTO' : sa.NVARCHAR(length=255) ,
'DT_NASC' : sa.NVARCHAR(length=255) ,
'ID_CLIENTE' : sa.INTEGER ,
'DT_PEDIDO' : sa.NVARCHAR(length=255) ,
'DATE_ID' : sa.NVARCHAR(length=255) ,
'METODOLOGIA' : sa.NVARCHAR(length=255) ,
'NR_LOJISTA' : sa.INTEGER ,
'LOJISTA' : sa.NVARCHAR(length=255) ,
'DS_TIPO_ENTREGA': sa.NVARCHAR(length=255),
'DATA_PONTO' : sa.NVARCHAR(length=255), 
'SEXO' : sa.NVARCHAR(length=255), 
'NOTA' : sa.INTEGER, 
'UF' : sa.NVARCHAR(length=255), 
'DEPTO' : sa.NVARCHAR(length=255), 
'SETOR' : sa.NVARCHAR(length=255), 
'VALOR_PEDIDO' : sa.NVARCHAR(length=255),
'QTD_PRODUTOS' :sa.INTEGER,
'ID_COMPRA' :sa.INTEGER}

ISC_ALL.to_sql('vv_cadastro',engine_OUT,schema=None,if_exists='replace', chunksize=500, index=False,dtype=dtype1)

print('Done uploading cadastro')

BANDEIRA_NUM_PEDIDO
0        138719186EX
1        138526003EX
2        138881829PF
3        137387037CB
4        138863319EX
5        138867413PF
6        138928179EX
7        138721143EX
8        138928232PF
9        138509302PF
10       138533772PF
11       138586799CB
12       138352465CB
13       138491016PF
14       138719659PF
15       138113392EX
16       138746838CB
17       138556234CB
18       138767972PF
19       138726083PF
20       138815447CB
21       139043596CB
22       138830800EX
23       138737224EX
24       138868925CB
25       138804780EX
26       138731240EX
27       138571425CB
28       138593580EX
29       138855319EX
            ...     
43086    137233409CB
43087    137352220CB
43088    137254100CB
43089    137180252CB
43090    137344919EX
43091    136288982EX
43092    138127860EX
43093    138083662PF
43094    136967467EX
43095    136938877EX
43096    137123923EX
43097    137125443EX
43098    136959119CB
43099    137340032EX
43100    137174722CB
43101    13733

Done uploading cadastro


<b><h1>Treating the dates</b></h1>

In [26]:
#Converting interview date to str
BASE_TOLUNA['DATA'] = BASE_TOLUNA['INTERVIEW_DATE'].astype('str')

#Creating ANO, MES and DIA
BASE_TOLUNA['ANO'] = BASE_TOLUNA['DATA'].str[:4]
BASE_TOLUNA['MES'] = BASE_TOLUNA['DATA'].str[4:6]
BASE_TOLUNA['DIA'] = BASE_TOLUNA['DATA'].str[-2:]

#Concatenating the columns to create the dates
BASE_TOLUNA['DATE_ID'] = BASE_TOLUNA['DIA']+"/"+BASE_TOLUNA['MES']+"/"+BASE_TOLUNA['ANO']

del(BASE_TOLUNA['MES'])
del(BASE_TOLUNA['DIA'])
del(BASE_TOLUNA['ANO'])
del(BASE_TOLUNA['DATA'])
del(BASE_TOLUNA['INTERVIEW_DATE'])

#Removing duplicates from Toluna base
# BASE_TOLUNA = BASE_TOLUNA.drop_duplicates(subset=['BANDEIRA_NUM_PEDIDO'], keep='first')

BASE_TOLUNA

,RESPID,ID_QUESTIONARIO,Q0,Q1,Q2,Q3,Q4,Q5,Q6,Q7,...,TIPO_LISTA,QUESTION_QUOTA,BANDEIRA,NUM_PEDIDO,BANDEIRA_NUM_PEDIDO,NUM_CLIENTE,NOME_SEGMENTO,TIPO_EMTREGA,TIPO_LISTA.1,DATE_ID
0,1448,4,7,6,6,8,8,8,8,NaN,...,,1,CB,129479522,129479522CB,3353125,MKTPLACE,CLIENTE,,08/11/2017
1,1450,1,0,1,1,6,0,6,0,NaN,...,,1,CB,130786955,130786955CB,1789527,B2C,CLIENTE,,08/11/2017
2,1451,5,10,9,9,9,9,9,9,9.0,...,,1,CB,129707220,129707220CB,5339896,MISTO,CLIENTE,,08/11/2017
3,1452,1,10,3,8,10,9,8,10,NaN,...,,1,CB,130508896,130508896CB,6944701,B2C,CLIENTE,,08/11/2017
4,1454,1,10,10,10,10,10,10,10,NaN,...,,1,CB,130488240,130488240CB,5003719,B2C,CLIENTE,,08/11/2017
5,1455,1,8,7,6,8,8,8,8,NaN,...,,1,CB,129905910,129905910CB,5875270,B2C,CLIENTE,,08/11/2017
6,1458,1,9,8,8,8,8,8,8,NaN,...,,1,CB,130291376,130291376CB,15465538,B2C,CLIENTE,,08/11/2017
7,1459,1,10,9,9,9,10,9,10,NaN,...,,1,CB,130449013,130449013CB,8602998,B2C,CLIENTE,,08/11/2017
8,1461,4,10,10,10,10,10,10,10,NaN,...,,1,CB,128700920,128700920CB,15337596,MKTPLACE,CLIENTE,,08/11/2017
9,1462,1,5,7,6,7,6,7,6,NaN,...,,1,CB,130458355,130458355CB,15165308,B2C,CLIENTE,,08/11/2017


<b><h1>RETRIEVING LY INFO AND LEFT JOIN BASE TOLUNA ON ISC ALL AND LEFT ON ISC VALUES</b></h1>

In [6]:
    BASE_LY="""SELECT 

    'NPS' AS DIMENSAO,
    'Em uma escala de 0 a 10, o quanto voce recomendaria nossa empresa para um amigo?' AS ATRIBUTO,
    DATE_ID,
    SUBSTR(date_id,4,2)||'/'||SUBSTR(date_id,9,2) as DATA_ENTREVISTA,
    CASE WHEN SUBSTR(date_id,7,4) = '2016' THEN 2016
         WHEN SUBSTR(date_id,7,4) = '2017' THEN 2017
         WHEN SUBSTR(date_id,7,4) = '2018' THEN 2018 END AS ANO,


    ID_COMPRA,
    BANDEIRA,
    ID_SEGMENTO,
    SEGMENTO as QUESTIONARIO,
    METODOLOGIA,
    '' as FLAG_PERIODO,
    '' as NR_LOJISTA,
    '' as LOJISTA,
    '' AS SEXO,
    '' as IDADE,
    '' as UF,
    '' as DEPTO,
    '' as SETOR,
    DS_TIPO_ENTREGA,
    Q0 AS VALOR

    FROM AN_RT_WS36.VV_LY_163

    WHERE Q0>= 0"""
BASE_LY = pd.read_sql(BASE_LY, engine_OUT)
BASE_LY.columns = map(unicode.upper, BASE_LY.columns)
# BASE_LY.rename(columns={'SEGMENTO':'NOME_SEGMENTO'}, inplace=True)
BASE_LY

,DIMENSAO,ATRIBUTO,DATE_ID,DATA_ENTREVISTA,ANO,ID_COMPRA,BANDEIRA,ID_SEGMENTO,QUESTIONARIO,METODOLOGIA,FLAG_PERIODO,NR_LOJISTA,LOJISTA,SEXO,IDADE,UF,DEPTO,SETOR,DS_TIPO_ENTREGA,VALOR
0,NPS,"Em uma escala de 0 a 10, o quanto voce recomen...",31/10/2017,10/17,2017,129501883,PF,6,MULTICANALIDADE,None,None,None,None,None,None,None,None,None,Estoque Cnova,10
1,NPS,"Em uma escala de 0 a 10, o quanto voce recomen...",01/11/2017,11/17,2017,129682443,CB,6,MULTICANALIDADE,None,None,None,None,None,None,None,None,None,Estoque Cnova,10
2,NPS,"Em uma escala de 0 a 10, o quanto voce recomen...",06/11/2017,11/17,2017,129507394,PF,1,B2C,None,None,None,None,None,None,None,None,None,Normal,9
3,NPS,"Em uma escala de 0 a 10, o quanto voce recomen...",03/11/2017,11/17,2017,129688554,CB,1,B2C,None,None,None,None,None,None,None,None,None,Normal,10
4,NPS,"Em uma escala de 0 a 10, o quanto voce recomen...",31/10/2017,10/17,2017,129691110,EX,1,B2C,None,None,None,None,None,None,None,None,None,Normal,9
5,NPS,"Em uma escala de 0 a 10, o quanto voce recomen...",28/10/2017,10/17,2017,129749783,EX,6,MULTICANALIDADE,None,None,None,None,None,None,None,None,None,Estoque Cnova,9
6,NPS,"Em uma escala de 0 a 10, o quanto voce recomen...",02/11/2017,11/17,2017,129917644,EX,1,B2C,None,None,None,None,None,None,None,None,None,Estoque Cnova,7
7,NPS,"Em uma escala de 0 a 10, o quanto voce recomen...",05/11/2017,11/17,2017,130223036,CB,1,B2C,None,None,None,None,None,None,None,None,None,Normal,10
8,NPS,"Em uma escala de 0 a 10, o quanto voce recomen...",25/10/2017,10/17,2017,129035337,CB,6,MULTICANALIDADE,None,None,None,None,None,None,None,None,None,Estoque Cnova,0
9,NPS,"Em uma escala de 0 a 10, o quanto voce recomen...",01/11/2017,11/17,2017,129038700,PF,1,B2C,None,None,None,None,None,None,None,None,None,Normal,10


In [85]:
dtype1={
'RESPID' : sa.INTEGER, 
'RANK' : sa.NVARCHAR(length=255) ,
'QUESTIONARIO' : sa.NVARCHAR(length=255) ,
'DIMENSAO' : sa.NVARCHAR(length=255) ,
'ATRIBUTO' : sa.NVARCHAR(length=255) ,
'SEGMENTO' : sa.NVARCHAR(length=255) ,    
'ID_QUESTIONARIO' : sa.INTEGER ,
'RESPSTATUS' : sa.INTEGER ,
'CAPTURE_LINK' : sa.NVARCHAR(length=255) ,
'ID_BANDEIRA' : sa.INTEGER ,
'ID_SEGMENTO' : sa.INTEGER ,
'ID_ENTREGA' : sa.INTEGER ,
'TIPO_LISTA' : sa.NVARCHAR(length=255),
'TIPO_LISTA.1' : sa.NVARCHAR(length=255),
'QUESTION_QUOTA' : sa.INTEGER ,
'BANDEIRA' : sa.NVARCHAR(length=255) ,
'FLAG_PERIODO' : sa.NVARCHAR(length=255) ,
'NUM_PEDIDO' : sa.NVARCHAR(length=255) ,
'BANDEIRA_NUM_PEDIDO' : sa.NVARCHAR(length=255) ,
'DS_TIPO_ENTREGA' : sa.NVARCHAR(length=255) ,
'TIPO_EMTREGA' : sa.NVARCHAR(length=255) ,    
'NOME_SEGMENTO' : sa.NVARCHAR(length=255) ,
'TIPO_ENTREGA' : sa.NVARCHAR(length=255) ,
'DATE_ID' : sa.NVARCHAR(length=255) ,
'DT_NASC' : sa.NVARCHAR(length=255) ,    
'DATA_ENTREVISTA' : sa.NVARCHAR(length=255) ,
'ANO' : sa.INTEGER ,
'NUM_CLIENTE' : sa.INTEGER ,
'ID_CLIENTE' : sa.INTEGER,
'ID_COMPRA' :sa.INTEGER,
'LOJISTA' :sa.NVARCHAR(length=255),
'ENT' : sa.NVARCHAR(length=255) ,
'DT_PEDIDO' : sa.NVARCHAR(length=255) ,
'PONTO_CONTROLE': sa.NVARCHAR(length=255),
'DATA_PONTO' : sa.NVARCHAR(length=255), 
'SEXO' : sa.NVARCHAR(length=255), 
'IDADE' : sa.NVARCHAR(length=255), 
'UF' : sa.NVARCHAR(length=255), 
'DEPTO' : sa.NVARCHAR(length=255), 
'SETOR' : sa.NVARCHAR(length=255), 
'METODOLOGIA' : sa.NVARCHAR(length=255),
'NOTA' : sa.INTEGER,   
'Q0' : sa.INTEGER,
'Q1' : sa.INTEGER,
'Q2' : sa.INTEGER,
'Q3' : sa.INTEGER,
'Q4' : sa.INTEGER,
'Q5' : sa.INTEGER,
'Q6' : sa.INTEGER,
'Q7' : sa.INTEGER,
'Q8' : sa.INTEGER,
'Q9' : sa.INTEGER,
'Q10' : sa.INTEGER,
'Q11' : sa.INTEGER,
'Q12' : sa.INTEGER,
'Q13' : sa.INTEGER,
'Q14' : sa.INTEGER,
'Q15' : sa.INTEGER,
'Q16' : sa.INTEGER,
'Q17' : sa.INTEGER,
'Q18' : sa.INTEGER,
'Q19' : sa.INTEGER,
'Q20' : sa.INTEGER,
'Q21' : sa.INTEGER,
'Q22' : sa.INTEGER,
'Q23' : sa.INTEGER,
'Q24' : sa.INTEGER,
'Q25' : sa.INTEGER,
'Q26' : sa.INTEGER,
'Q27' : sa.INTEGER,
'Q28' : sa.INTEGER,
'Q29' : sa.INTEGER,
'Q30' : sa.INTEGER,
'Q31' : sa.INTEGER,
'Q32' : sa.INTEGER,
'Q33' : sa.INTEGER,
'Q34' : sa.INTEGER,
'Q35' : sa.INTEGER,
'Q36' : sa.INTEGER,
'Q37' : sa.INTEGER,
'Q38' : sa.INTEGER,
'Q39' : sa.INTEGER,
'Q40' : sa.INTEGER,
'Q41' : sa.INTEGER,
'VALOR' : sa.INTEGER,
'ISC' : sa.INTEGER}


# BASE_LY['ID_QUESTIONARIO'] = BASE_LY['ID_SEGMENTO']
BASE_TOLUNA['FLAG_PERIODO'] = 'CY'
RAW_DATA_FULL = pd.merge(BASE_TOLUNA, CADASTRO_HIST, how='left', on=['BANDEIRA_NUM_PEDIDO'])
RAW_DATA_FULL = pd.merge(RAW_DATA_FULL, ISC_VALOR, how='left', on=['RESPID'])
# RAW_DATA_FULL = pd.merge(RAW_DATA_FULL, METODOLOGIA, how='left', on=['RESPID'])
# RAW_DATA_FULL['BANDEIRA'] = RAW_DATA_FULL['BANDEIRA_x']
# RAW_DATA_FULL['TIPO_LISTA'] = RAW_DATA_FULL['TIPO_LISTA_x']

# del(RAW_DATA_FULL['BANDEIRA_y'])
# del(RAW_DATA_FULL['BANDEIRA_x'])
# del(RAW_DATA_FULL['TIPO_LISTA_x'])
# del(RAW_DATA_FULL['TIPO_LISTA_y'])
del(RAW_DATA_FULL['CAPTURE_LINK'])
del(RAW_DATA_FULL['RESPSTATUS'])
del(RAW_DATA_FULL['NUM_CLIENTE'])
del(RAW_DATA_FULL['NUM_PEDIDO'])
# del(RAW_DATA_FULL['ID_CLIENTE'])
del(RAW_DATA_FULL['QUESTION_QUOTA'])
# del(RAW_DATA_FULL['LOJISTA'])
del(RAW_DATA_FULL['TIPO_LISTA'])
del(RAW_DATA_FULL['TIPO_LISTA.1'])
# del(RAW_DATA_FULL['DATA_PONTO'])
# del(RAW_DATA_FULL['ENT'])

RAW_DATA_FULL = RAW_DATA_FULL.fillna(0) #<--- IMPORTANTE NÃO APAGAR PARA NÃO CAGAR O VALOR DO ISC!!!!!#
dtypedict = sqlcol(RAW_DATA_FULL) 
for column_name, column in RAW_DATA_FULL.transpose().iterrows():
    print column_name
    print(RAW_DATA_FULL[column_name])
    RAW_DATA_FULL[column_name] = RAW_DATA_FULL[column_name].astype('str')
    
RAW_DATA_FULL.to_sql('raw_data_full_new',engine_OUT,schema=None,if_exists='replace', chunksize=500, index=False,dtype={col_name: NVARCHAR(255) for col_name in RAW_DATA_FULL})
RAW_DATA_FULL

RESPID
0          1448
1          1450
2          1451
3          1452
4          1454
5          1455
6          1458
7          1459
8          1461
9          1462
10         1463
11         1464
12         1465
13         1466
14         1467
15         1469
16         1470
17         1471
18         1472
19         1479
20         1480
21         1481
22         1482
23         1483
24         1485
25         1487
26         1488
27         1489
28         1490
29         1491
          ...  
76587    231617
76588    231619
76589    231627
76590    231629
76591    231635
76592    231643
76593    231646
76594    231651
76595    231658
76596    231659
76597    231660
76598    231666
76599    231667
76600    231673
76601    231677
76602    231678
76603    231684
76604    231690
76605    231691
76606    231693
76607    231695
76608    231700
76609    231701
76610    231708
76611    231712
76612    231714
76613    231726
76614    231729
76615    231735
76616    231737
Name: RESPID, Len

Q7
0        0.0
1        0.0
2        9.0
3        0.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       0.0
11       0.0
12       0.0
13       0.0
14       0.0
15       0.0
16       0.0
17       0.0
18       0.0
19       0.0
20       0.0
21       0.0
22       0.0
23       0.0
24       0.0
25       0.0
26       0.0
27       0.0
28       0.0
29       0.0
        ... 
76587    0.0
76588    0.0
76589    0.0
76590    0.0
76591    0.0
76592    0.0
76593    0.0
76594    0.0
76595    0.0
76596    0.0
76597    0.0
76598    0.0
76599    0.0
76600    0.0
76601    0.0
76602    0.0
76603    0.0
76604    0.0
76605    0.0
76606    0.0
76607    0.0
76608    0.0
76609    0.0
76610    0.0
76611    0.0
76612    0.0
76613    0.0
76614    0.0
76615    0.0
76616    0.0
Name: Q7, Length: 76617, dtype: float64
Q8
0        0.0
1        0.0
2        5.0
3        0.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       0.0
11       0.0
12   

Q19
0         7
1         6
2         9
3         9
4        10
5         8
6         8
7        10
8        10
9         6
10        9
11        7
12        8
13        1
14        9
15       10
16       10
17       10
18       10
19       10
20        9
21        9
22        9
23        2
24        8
25       10
26        8
27        8
28        9
29        9
         ..
76587    10
76588     9
76589     8
76590     9
76591     9
76592    10
76593    10
76594     9
76595     8
76596     8
76597     8
76598    10
76599     8
76600     4
76601     8
76602     6
76603    10
76604     9
76605     8
76606    10
76607     9
76608     9
76609     7
76610     7
76611    10
76612     7
76613     4
76614    10
76615    10
76616     9
Name: Q19, Length: 76617, dtype: int64
Q20
0         7
1         0
2        10
3         9
4        10
5         9
6         7
7        10
8        10
9         7
10       10
11        7
12       10
13        5
14        9
15       10
16       10
17       10
18   

Q30
0         7
1        10
2         5
3        10
4        10
5        10
6         9
7        10
8         9
9         6
10        0
11        7
12       10
13        1
14        9
15       10
16       10
17       10
18       10
19       10
20        0
21       10
22        9
23        0
24       10
25       10
26       10
27        0
28       10
29        0
         ..
76587    10
76588     9
76589    10
76590    10
76591     9
76592    10
76593    10
76594    10
76595     9
76596    10
76597     4
76598    10
76599     0
76600     0
76601    10
76602     0
76603    10
76604     9
76605     0
76606    10
76607    10
76608     0
76609     1
76610     7
76611     0
76612     0
76613    10
76614    10
76615    10
76616     9
Name: Q30, Length: 76617, dtype: int64
Q31
0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
5         0.0
6         0.0
7         0.0
8        10.0
9         0.0
10        0.0
11        7.0
12        0.0
13        0.0
14        0.0
15        0

Q40
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       0.0
11       0.0
12       0.0
13       0.0
14       0.0
15       0.0
16       0.0
17       0.0
18       0.0
19       0.0
20       0.0
21       0.0
22       0.0
23       0.0
24       0.0
25       0.0
26       0.0
27       0.0
28       0.0
29       0.0
        ... 
76587    0.0
76588    0.0
76589    0.0
76590    0.0
76591    0.0
76592    0.0
76593    0.0
76594    0.0
76595    0.0
76596    0.0
76597    0.0
76598    0.0
76599    0.0
76600    0.0
76601    0.0
76602    0.0
76603    0.0
76604    0.0
76605    0.0
76606    0.0
76607    0.0
76608    0.0
76609    0.0
76610    0.0
76611    0.0
76612    0.0
76613    0.0
76614    0.0
76615    0.0
76616    0.0
Name: Q40, Length: 76617, dtype: float64
Q41
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       0.0
11       0.0
12

DEPTO
0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
76587    0
76588    0
76589    0
76590    0
76591    0
76592    0
76593    0
76594    0
76595    0
76596    0
76597    0
76598    0
76599    0
76600    0
76601    0
76602    0
76603    0
76604    0
76605    0
76606    0
76607    0
76608    0
76609    0
76610    0
76611    0
76612    0
76613    0
76614    0
76615    0
76616    0
Name: DEPTO, Length: 76617, dtype: object
METODOLOGIA
0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24   

,RESPID,INTERVIEW_DATE,ID_QUESTIONARIO,Q0,Q1,Q2,Q3,Q4,Q5,Q6,...,FLAG_PERIODO,DS_TIPO_ENTREGA,DATE_ID,DEPTO,METODOLOGIA,SETOR,UF,DT_NASC,SEXO,ISC
0,1448,20171108,4,7,6,6,8,8,8,8,...,CY,0,0,0,0,0,0,0,0,56.907181955
1,1450,20171108,1,0,1,1,6,0,6,0,...,CY,0,0,0,0,0,0,0,0,43.434616487
2,1451,20171108,5,10,9,9,9,9,9,9,...,CY,0,0,0,0,0,0,0,0,76.0418909604
3,1452,20171108,1,10,3,8,10,9,8,10,...,CY,0,0,0,0,0,0,0,0,91.3165489263
4,1454,20171108,1,10,10,10,10,10,10,10,...,CY,0,0,0,0,0,0,0,0,90.4426178453
5,1455,20171108,1,8,7,6,8,8,8,8,...,CY,0,0,0,0,0,0,0,0,85.7906963795
6,1458,20171108,1,9,8,8,8,8,8,8,...,CY,0,0,0,0,0,0,0,0,80.4289040465
7,1459,20171108,1,10,9,9,9,10,9,10,...,CY,0,0,0,0,0,0,0,0,96.6562108648
8,1461,20171108,4,10,10,10,10,10,10,10,...,CY,0,0,0,0,0,0,0,0,96.5032667402
9,1462,20171108,1,5,7,6,7,6,7,6,...,CY,0,0,0,0,0,0,0,0,64.0704582602


# Treating the DT_NASC variable

In [87]:
raw_data_full_new2="""

SELECT DISTINCT
RESPID,
ID_QUESTIONARIO,
Q0,
Q1,
Q2,
Q3,
Q4,
Q5,
Q6,
Q7,
Q8,
Q9,
Q10,
Q11,
Q12,
Q13,
Q14,
Q15,
Q16,
Q17,
Q18,
Q19,
Q20,
Q21,
Q22,
Q23,
Q24,
Q25,
Q26,
Q27,
Q28,
Q29,
Q30,
Q31,
Q32,
Q33,
Q34,
Q35,
Q36,
Q37,
Q38,
Q39,
Q40,
Q41,
ID_BANDEIRA,
ID_SEGMENTO,
ID_ENTREGA,
BANDEIRA_NUM_PEDIDO,
NOME_SEGMENTO,
TIPO_EMTREGA,
DATE_ID as DATE_ID,
FLAG_PERIODO,
DEPTO,
DS_TIPO_ENTREGA,
CASE WHEN DT_NASC = '0' THEN NULL ELSE DT_NASC END AS DT_NASC2,
SETOR,
SEXO,
UF,
ISC,
METODOLOGIA,
BANDEIRA

FROM AN_RT_WS36.RAW_DATA_FULL_NEW"""
raw_data_full_new2 = pd.read_sql(raw_data_full_new2, engine_OUT)
raw_data_full_new2.to_sql('raw_data_full_new2',engine_OUT,schema=None,if_exists='replace', chunksize=500, index=False,dtype={col_name: NVARCHAR(255) for col_name in raw_data_full_new2})
raw_data_full_new2

,respid,id_questionario,q0,q1,q2,q3,q4,q5,q6,q7,...,flag_periodo,depto,ds_tipo_entrega,dt_nasc2,setor,sexo,uf,isc,metodologia,bandeira
0,82541,1,8,8,7,8,8,6,8,0.0,...,CY,0,0,None,0,0,0,80.5036207713,0,EX
1,82561,6,4,7,7,7,7,7,7,0.0,...,CY,0,0,None,0,0,0,68.5980272138,0,PF
2,82630,4,9,9,9,7,7,7,7,0.0,...,CY,0,0,None,0,0,0,74.687905938,0,PF
3,82709,4,0,7,5,7,7,7,7,0.0,...,CY,0,0,None,0,0,0,41.5868201249,0,EX
4,82784,4,8,8,8,7,7,7,8,0.0,...,CY,0,0,None,0,0,0,72.645649712,0,CB
5,82876,6,10,10,10,10,10,10,10,0.0,...,CY,0,0,None,0,0,0,100.0,0,PF
6,82895,4,0,4,4,4,4,4,4,0.0,...,CY,0,0,None,0,0,0,21.9939371663,0,EX
7,82922,1,8,8,8,8,8,8,9,0.0,...,CY,0,0,None,0,0,0,80.3284693682,0,EX
8,83094,1,8,2,8,3,2,2,9,0.0,...,CY,0,0,None,0,0,0,39.8899444379,0,CB
9,83350,4,5,8,6,7,7,4,10,0.0,...,CY,0,0,None,0,0,0,78.216039651,0,EX


<b><h1>SEPARATING EACH QUESTION FOR EACH QUESTIONARIE</b></h1>

In [7]:
def processa_base(questionario,dimensao,atributo,pergunta,id_questionario):
    SQL1="""
    SELECT DISTINCT 
    '{questionario}' AS QUESTIONARIO,
    '{dimensao}' AS DIMENSAO,
    '{atributo}' AS ATRIBUTO, 
    FLAG_PERIODO,
    RESPID,
    /*METODOLOGIA,*/
    ID_QUESTIONARIO,
    /*RESPSTATUS,*/
   /*CAPTURE_LINK,*/
    ID_BANDEIRA,
    ID_SEGMENTO,
    ID_ENTREGA,
    /*TIPO_LISTA,*/
    /*QUESTION_QUOTA,*/
    BANDEIRA,
    /*ID_COMPRA AS NUM_PEDIDO,*/
    BANDEIRA_NUM_PEDIDO,
    /*NUM_CLIENTE,*/
    NOME_SEGMENTO,
    TIPO_EMTREGA as TIPO_ENTREGA,
    DS_TIPO_ENTREGA,
    DATE_ID,
    CASE WHEN (SUBSTR(date_id,4,2)||'/'||SUBSTR(date_id,9,2)) = '' THEN NULL ELSE SUBSTR(date_id,4,2)||'/'||SUBSTR(date_id,9,2) END as DATA_ENTREVISTA,
    CASE WHEN SUBSTR(date_id,7,4) = '2016' THEN 2016
         WHEN SUBSTR(date_id,7,4) = '2017' THEN 2017
         WHEN SUBSTR(date_id,7,4) = '2018' THEN 2018 END AS ANO,
    /*ID_CLIENTE,*/
    /*ENT,*/
    /*DT_PEDIDO AS DATA_PEDIDO,*/
    /*NR_ENTREGA AS N_ENTREGA,*/
    /*NR_LOJISTA AS NUM_LOJISTA,*/
    /*LOJISTA,*/
    /*PONTO_CONTROLE,*/
    /*DATA_PONTO,*/
    CASE WHEN SEXO IN ('m','M') THEN 'M'
         WHEN SEXO IN ('f','F') THEN 'F' END AS SEXO,
    CASE 
         WHEN ROUND(months_between(TRUNC(sysdate),to_date(DT_NASC2,'DD-MM-YYYY'))/12) IN ('','0', '32') THEN ''
         WHEN ROUND(months_between(TRUNC(sysdate),to_date(DT_NASC2,'DD-MM-YYYY'))/12) BETWEEN '0'  AND '17' THEN '< 18'
         WHEN ROUND(months_between(TRUNC(sysdate),to_date(DT_NASC2,'DD-MM-YYYY'))/12) BETWEEN '18' AND '24' THEN '18 - 24'
         WHEN ROUND(months_between(TRUNC(sysdate),to_date(DT_NASC2,'DD-MM-YYYY'))/12) BETWEEN '25' AND '35' THEN '25 - 35'
         WHEN ROUND(months_between(TRUNC(sysdate),to_date(DT_NASC2,'DD-MM-YYYY'))/12) BETWEEN '36' AND '45' THEN '36 - 45'
         WHEN ROUND(months_between(TRUNC(sysdate),to_date(DT_NASC2,'DD-MM-YYYY'))/12) BETWEEN '46' AND '55' THEN '46 - 55'
         WHEN ROUND(months_between(TRUNC(sysdate),to_date(DT_NASC2,'DD-MM-YYYY'))/12) BETWEEN '56' AND '65' THEN '56 - 65'
         WHEN ROUND(months_between(TRUNC(sysdate),to_date(DT_NASC2,'DD-MM-YYYY'))/12) >= '65' THEN '65+' END as IDADE,
    UF,
    DEPTO,
    SETOR,
    /*VALOR_PEDIDO,*/
    /*QTD_PRODUTOS,*/
    AVG(ISC) AS ISC,
    AVG({pergunta}) as VALOR


    FROM raw_data_full_new2

    WHERE  ID_QUESTIONARIO = {id_questionario} AND {pergunta}>=0
    
    GROUP BY 
    
    FLAG_PERIODO,
    RESPID,
    /*METODOLOGIA,*/
    ID_QUESTIONARIO,
    /*RESPSTATUS,*/
    /*CAPTURE_LINK,*/
    ID_BANDEIRA,
    ID_SEGMENTO,
    ID_ENTREGA,
    /*TIPO_LISTA,*/
    /*QUESTION_QUOTA,*/
    BANDEIRA,
    /*ID_COMPRA,*/
    BANDEIRA_NUM_PEDIDO,
    /*NUM_CLIENTE,*/
    NOME_SEGMENTO,
    TIPO_EMTREGA,
    DS_TIPO_ENTREGA,
    DATE_ID,
    SUBSTR(date_id,4,2)||'/'||SUBSTR(date_id,9,2),
    CASE WHEN SUBSTR(date_id,7,4) = '2016' THEN 2016
         WHEN SUBSTR(date_id,7,4) = '2017' THEN 2017
         WHEN SUBSTR(date_id,7,4) = '2018' THEN 2018 END,
     /*ID_CLIENTE,*/
    /*ENT,*/
    /*DT_PEDIDO,*/
    /*NR_ENTREGA,*/
    /*NR_LOJISTA,*/
     /*LOJISTA,*/
    /*PONTO_CONTROLE,*/
    /*DATA_PONTO,*/
    CASE WHEN SEXO IN ('m','M') THEN 'M'
         WHEN SEXO IN ('f','F') THEN 'F' END,
    CASE 
         WHEN ROUND(months_between(TRUNC(sysdate),to_date(DT_NASC2,'DD-MM-YYYY'))/12) IN ('','0','32') THEN ''
         WHEN ROUND(months_between(TRUNC(sysdate),to_date(DT_NASC2,'DD-MM-YYYY'))/12) BETWEEN '0'  AND '17' THEN '< 18'
         WHEN ROUND(months_between(TRUNC(sysdate),to_date(DT_NASC2,'DD-MM-YYYY'))/12) BETWEEN '18' AND '24' THEN '18 - 24'
         WHEN ROUND(months_between(TRUNC(sysdate),to_date(DT_NASC2,'DD-MM-YYYY'))/12) BETWEEN '25' AND '35' THEN '25 - 35'
         WHEN ROUND(months_between(TRUNC(sysdate),to_date(DT_NASC2,'DD-MM-YYYY'))/12) BETWEEN '36' AND '45' THEN '36 - 45'
         WHEN ROUND(months_between(TRUNC(sysdate),to_date(DT_NASC2,'DD-MM-YYYY'))/12) BETWEEN '46' AND '55' THEN '46 - 55'
         WHEN ROUND(months_between(TRUNC(sysdate),to_date(DT_NASC2,'DD-MM-YYYY'))/12) BETWEEN '56' AND '65' THEN '56 - 65'
         WHEN ROUND(months_between(TRUNC(sysdate),to_date(DT_NASC2,'DD-MM-YYYY'))/12) >= '65' THEN '65+' END,
    UF,
    DEPTO,
    SETOR""".format(questionario=questionario,dimensao=dimensao,atributo=atributo,pergunta=pergunta,id_questionario=id_questionario)
    return SQL1

In [8]:
# Getting the preriods
BASE_B2C_1 = pd.read_sql(processa_base("B2C",'NPS','Em uma escala de 0 a 10, o quanto voce recomendaria nossa empresa para um amigo?','Q0',1), engine_OUT)
BASE_B2C_2 = pd.read_sql(processa_base("B2C",'PRECOS_OFERTAS','Preco dos produtos em geral','Q1',1), engine_OUT)
BASE_B2C_3 = pd.read_sql(processa_base("B2C",'PRECOS_OFERTAS','Atratividade das ofertas em nosso site','Q2',1), engine_OUT)
BASE_B2C_4 = pd.read_sql(processa_base("B2C",'NAVEGACAO','Facilidade para pesquisar e encontrar produtos pelo menu de navegacao','Q3',1), engine_OUT)
BASE_B2C_5 = pd.read_sql(processa_base("B2C",'NAVEGACAO','Facilidade em reconhecer as palavras na busca por um produto','Q4',1), engine_OUT)
BASE_B2C_6 = pd.read_sql(processa_base("B2C",'NAVEGACAO','Facilidade de comparar precos e produtos similares no site','Q5',1), engine_OUT)
BASE_B2C_7 = pd.read_sql(processa_base("B2C",'NAVEGACAO','Facilidade na conclusao da compra','Q6',1), engine_OUT)
BASE_B2C_8 = pd.read_sql(processa_base("B2C",'NAVEGACAO','Opcao de compra de uma Loja Parceira em nosso site','Q7',1), engine_OUT)
BASE_B2C_9 = pd.read_sql(processa_base("B2C",'NAVEGACAO','Seguranca em comprar de uma Loja Parceira','Q8',1), engine_OUT)
BASE_B2C_10 = pd.read_sql(processa_base("B2C",'NAVEGACAO','Facilidade para pesquisar e encontrar a Lista de Casamento','Q9',1), engine_OUT)
BASE_B2C_11 = pd.read_sql(processa_base("B2C",'NAVEGACAO','Facilidade em escolher o produto','Q10',1), engine_OUT)
BASE_B2C_12 = pd.read_sql(processa_base("B2C",'NAVEGACAO','Facilidade para escrever a mensagem aos noivos','Q11',1), engine_OUT)
BASE_B2C_13 = pd.read_sql(processa_base("B2C",'NAVEGACAO','Facilidade para concluir a compra','Q12',1), engine_OUT)
BASE_B2C_14 = pd.read_sql(processa_base("B2C",'NAVEGACAO','Facilidade para criar e editar a Lista de Casamento','Q13',1), engine_OUT)
BASE_B2C_15 = pd.read_sql(processa_base("B2C",'NAVEGACAO','Facilidade para divulgar a sua Lista de Casamento','Q14',1), engine_OUT)
BASE_B2C_16 = pd.read_sql(processa_base("B2C",'NAVEGACAO','Facilidade para converter os produtos em creditos ou libera-los para entrega','Q15',1), engine_OUT)
BASE_B2C_17 = pd.read_sql(processa_base("B2C",'NAVEGACAO','Facilidade para agradecer aos convidados pelos presentes ganhos','Q16',1), engine_OUT)
BASE_B2C_18 = pd.read_sql(processa_base("B2C",'NAVEGACAO','Facilidade para acompanhar o saldo acumulado dos creditos','Q17',1), engine_OUT)
BASE_B2C_19 = pd.read_sql(processa_base("B2C",'PRODUTOS','Variedade de produtos e marcas','Q18',1), engine_OUT)
BASE_B2C_20 = pd.read_sql(processa_base("B2C",'PRODUTOS','Facilidade de encontrar informacoes sobre o produto (descricao detalhada, fotos e avaliacao de outros compradores)','Q19',1), engine_OUT)
BASE_B2C_21 = pd.read_sql(processa_base("B2C",'PAGAMENTO','Variedade de opcoes de pagamento','Q20',1), engine_OUT)
BASE_B2C_22 = pd.read_sql(processa_base("B2C",'PAGAMENTO','Clareza da taxa de juros em caso de parcelamento','Q21',1), engine_OUT)
BASE_B2C_23 = pd.read_sql(processa_base("B2C",'PAGAMENTO','Tempo de aprovacao do pagamento','Q22',1), engine_OUT)
BASE_B2C_24 = pd.read_sql(processa_base("B2C",'PAGAMENTO','Quantidade de parcelas sem juros','Q23',1), engine_OUT)
BASE_B2C_25 = pd.read_sql(processa_base("B2C",'PAGAMENTO','Clareza do valor do Frete','Q24',1), engine_OUT)
BASE_B2C_26 = pd.read_sql(processa_base("B2C",'PAGAMENTO','Opcao de compra utilizando vale','Q25',1), engine_OUT)
BASE_B2C_27 = pd.read_sql(processa_base("B2C",'ENTREGA','Clareza do valor do frete','Q26',1), engine_OUT)
BASE_B2C_28 = pd.read_sql(processa_base("B2C",'ENTREGA','Opcao de retirar a compra na loja','Q27',1), engine_OUT)
BASE_B2C_29 = pd.read_sql(processa_base("B2C",'ENTREGA','Acompanhamento da entrega da compra','Q28',1), engine_OUT)
BASE_B2C_30 = pd.read_sql(processa_base("B2C",'ENTREGA','Entrega dentro do prazo estipulado','Q29',1), engine_OUT)
BASE_B2C_31 = pd.read_sql(processa_base("B2C",'ENTREGA','Condicoes do produto entregue','Q30',1), engine_OUT)
BASE_B2C_32 = pd.read_sql(processa_base("B2C",'COMPRA_LOJA_PARCEIRA_MKT','Clareza e transparencia que a compra sera realizada em uma Loja Parceira','Q31',1), engine_OUT)
BASE_B2C_33 = pd.read_sql(processa_base("B2C",'COMPRA_LOJA_PARCEIRA_MKT','Seguranca em comprar de uma Loja Parceira','Q32',1), engine_OUT)
BASE_B2C_34 = pd.read_sql(processa_base("B2C",'RETIRADA_LOJA_FISICA','Variedade de lojas disponiveis para retirada','Q33',1), engine_OUT)
BASE_B2C_35 = pd.read_sql(processa_base("B2C",'RETIRADA_LOJA_FISICA','Prazo atrativo para retirada da compra na loja fisica','Q34',1), engine_OUT)
BASE_B2C_36 = pd.read_sql(processa_base("B2C",'RETIRADA_LOJA_FISICA','Informacoes claras sobre a retirada da compra','Q35',1), engine_OUT)
BASE_B2C_37 = pd.read_sql(processa_base("B2C",'RETIRADA_LOJA_FISICA','Acompanhamento da disponibilidade de retirada da compra','Q36',1), engine_OUT)
BASE_B2C_38 = pd.read_sql(processa_base("B2C",'RETIRADA_LOJA_FISICA','Facilidade, cordialidade e tempo de espera durante o atendimento na retirada da compra','Q37',1), engine_OUT)
BASE_B2C_39 = pd.read_sql(processa_base("B2C",'RETIRADA_LOJA_FISICA','Retirada dentro do prazo estipulado','Q38',1), engine_OUT)
BASE_B2C_40 = pd.read_sql(processa_base("B2C",'ATENDIMENTO','Facilidade para entrar em contato com nossa Central de Atendimento','Q39',1), engine_OUT)
BASE_B2C_41 = pd.read_sql(processa_base("B2C",'ATENDIMENTO','Atencao, cordialidade e conhecimento dos atendentes','Q40',1), engine_OUT)
BASE_B2C_42 = pd.read_sql(processa_base("B2C",'ATENDIMENTO','Prazo informado para resolucao','Q41',1), engine_OUT)
print('DONE B2C')

BASE_NOIVO_1 = pd.read_sql(processa_base("NOIVO",'NPS','Em uma escala de 0 a 10, o quanto voce recomendaria nossa empresa para um amigo?','Q0',2), engine_OUT)
BASE_NOIVO_2 = pd.read_sql(processa_base("NOIVO",'PRECOS_OFERTAS','Preco dos produtos em geral','Q1',2), engine_OUT)
BASE_NOIVO_3 = pd.read_sql(processa_base("NOIVO",'PRECOS_OFERTAS','Atratividade das ofertas em nosso site','Q2',2), engine_OUT)
BASE_NOIVO_4 = pd.read_sql(processa_base("NOIVO",'NAVEGACAO','Facilidade para pesquisar e encontrar produtos pelo menu de navegacao','Q3',2), engine_OUT)
BASE_NOIVO_5 = pd.read_sql(processa_base("NOIVO",'NAVEGACAO','Facilidade em reconhecer as palavras na busca por um produto','Q4',2), engine_OUT)
BASE_NOIVO_6 = pd.read_sql(processa_base("NOIVO",'NAVEGACAO','Facilidade de comparar precos e produtos similares no site','Q5',2), engine_OUT)
BASE_NOIVO_7 = pd.read_sql(processa_base("NOIVO",'NAVEGACAO','Facilidade na conclusao da compra','Q6',2), engine_OUT)
BASE_NOIVO_8 = pd.read_sql(processa_base("NOIVO",'NAVEGACAO','Opcao de compra de uma Loja Parceira em nosso site','Q7',2), engine_OUT)
BASE_NOIVO_9 = pd.read_sql(processa_base("NOIVO",'NAVEGACAO','Seguranca em comprar de uma Loja Parceira','Q8',2), engine_OUT)
BASE_NOIVO_10 = pd.read_sql(processa_base("NOIVO",'NAVEGACAO','Facilidade para pesquisar e encontrar a Lista de Casamento','Q9',2), engine_OUT)
BASE_NOIVO_11 = pd.read_sql(processa_base("NOIVO",'NAVEGACAO','Facilidade em escolher o produto','Q10',2), engine_OUT)
BASE_NOIVO_12 = pd.read_sql(processa_base("NOIVO",'NAVEGACAO','Facilidade para escrever a mensagem aos noivos','Q11',2), engine_OUT)
BASE_NOIVO_13 = pd.read_sql(processa_base("NOIVO",'NAVEGACAO','Facilidade para concluir a compra','Q12',2), engine_OUT)
BASE_NOIVO_14 = pd.read_sql(processa_base("NOIVO",'NAVEGACAO','Facilidade para criar e editar a Lista de Casamento','Q13',2), engine_OUT)
BASE_NOIVO_15 = pd.read_sql(processa_base("NOIVO",'NAVEGACAO','Facilidade para divulgar a sua Lista de Casamento','Q14',2), engine_OUT)
BASE_NOIVO_16 = pd.read_sql(processa_base("NOIVO",'NAVEGACAO','Facilidade para converter os produtos em creditos ou libera-los para entrega','Q15',2), engine_OUT)
BASE_NOIVO_17 = pd.read_sql(processa_base("NOIVO",'NAVEGACAO','Facilidade para agradecer aos convidados pelos presentes ganhos','Q16',2), engine_OUT)
BASE_NOIVO_18 = pd.read_sql(processa_base("NOIVO",'NAVEGACAO','Facilidade para acompanhar o saldo acumulado dos creditos','Q17',2), engine_OUT)
BASE_NOIVO_19 = pd.read_sql(processa_base("NOIVO",'PRODUTOS','Variedade de produtos e marcas','Q18',2), engine_OUT)
BASE_NOIVO_20 = pd.read_sql(processa_base("NOIVO",'PRODUTOS','Facilidade de encontrar informacoes sobre o produto (descricao detalhada, fotos e avaliacao de outros compradores)','Q19',2), engine_OUT)
BASE_NOIVO_21 = pd.read_sql(processa_base("NOIVO",'PAGAMENTO','Variedade de opcoes de pagamento','Q20',2), engine_OUT)
BASE_NOIVO_22 = pd.read_sql(processa_base("NOIVO",'PAGAMENTO','Clareza da taxa de juros em caso de parcelamento','Q21',2), engine_OUT)
BASE_NOIVO_23 = pd.read_sql(processa_base("NOIVO",'PAGAMENTO','Tempo de aprovacao do pagamento','Q22',2), engine_OUT)
BASE_NOIVO_24 = pd.read_sql(processa_base("NOIVO",'PAGAMENTO','Quantidade de parcelas sem juros','Q23',2), engine_OUT)
BASE_NOIVO_25 = pd.read_sql(processa_base("NOIVO",'PAGAMENTO','Clareza do valor do Frete','Q24',2), engine_OUT)
BASE_NOIVO_26 = pd.read_sql(processa_base("NOIVO",'PAGAMENTO','Opcao de compra utilizando vale','Q25',2), engine_OUT)
BASE_NOIVO_27 = pd.read_sql(processa_base("NOIVO",'ENTREGA','Clareza do valor do frete','Q26',2), engine_OUT)
BASE_NOIVO_28 = pd.read_sql(processa_base("NOIVO",'ENTREGA','Opcao de retirar a compra na loja','Q27',2), engine_OUT)
BASE_NOIVO_29 = pd.read_sql(processa_base("NOIVO",'ENTREGA','Acompanhamento da entrega da compra','Q28',2), engine_OUT)
BASE_NOIVO_30 = pd.read_sql(processa_base("NOIVO",'ENTREGA','Entrega dentro do prazo estipulado','Q29',2), engine_OUT)
BASE_NOIVO_31 = pd.read_sql(processa_base("NOIVO",'ENTREGA','Condicoes do produto entregue','Q30',2), engine_OUT)
BASE_NOIVO_32 = pd.read_sql(processa_base("NOIVO",'COMPRA_LOJA_PARCEIRA_MKT','Clareza e transparencia que a compra sera realizada em uma Loja Parceira','Q31',2), engine_OUT)
BASE_NOIVO_33 = pd.read_sql(processa_base("NOIVO",'COMPRA_LOJA_PARCEIRA_MKT','Seguranca em comprar de uma Loja Parceira','Q32',2), engine_OUT)
BASE_NOIVO_34 = pd.read_sql(processa_base("NOIVO",'RETIRADA_LOJA_FISICA','Variedade de lojas disponiveis para retirada','Q33',2), engine_OUT)
BASE_NOIVO_35 = pd.read_sql(processa_base("NOIVO",'RETIRADA_LOJA_FISICA','Prazo atrativo para retirada da compra na loja fisica','Q34',2), engine_OUT)
BASE_NOIVO_36 = pd.read_sql(processa_base("NOIVO",'RETIRADA_LOJA_FISICA','Informacoes claras sobre a retirada da compra','Q35',2), engine_OUT)
BASE_NOIVO_37 = pd.read_sql(processa_base("NOIVO",'RETIRADA_LOJA_FISICA','Acompanhamento da disponibilidade de retirada da compra','Q36',2), engine_OUT)
BASE_NOIVO_38 = pd.read_sql(processa_base("NOIVO",'RETIRADA_LOJA_FISICA','Facilidade, cordialidade e tempo de espera durante o atendimento na retirada da compra','Q37',2), engine_OUT)
BASE_NOIVO_39 = pd.read_sql(processa_base("NOIVO",'RETIRADA_LOJA_FISICA','Retirada dentro do prazo estipulado','Q38',2), engine_OUT)
BASE_NOIVO_40 = pd.read_sql(processa_base("NOIVO",'ATENDIMENTO','Facilidade para entrar em contato com nossa Central de Atendimento','Q39',2), engine_OUT)
BASE_NOIVO_41 = pd.read_sql(processa_base("NOIVO",'ATENDIMENTO','Atencao, cordialidade e conhecimento dos atendentes','Q40',2), engine_OUT)
BASE_NOIVO_42 = pd.read_sql(processa_base("NOIVO",'ATENDIMENTO','Prazo informado para resolucao','Q41',2), engine_OUT)
print('DONE NOIVO')

BASE_CONVIDADO_1 = pd.read_sql(processa_base("CONVIDADO",'NPS','Em uma escala de 0 a 10, o quanto voce recomendaria nossa empresa para um amigo?','Q0',3), engine_OUT)
BASE_CONVIDADO_2 = pd.read_sql(processa_base("CONVIDADO",'PRECOS_OFERTAS','Preco dos produtos em geral','Q1',3), engine_OUT)
BASE_CONVIDADO_3 = pd.read_sql(processa_base("CONVIDADO",'PRECOS_OFERTAS','Atratividade das ofertas em nosso site','Q2',3), engine_OUT)
BASE_CONVIDADO_4 = pd.read_sql(processa_base("CONVIDADO",'NAVEGACAO','Facilidade para pesquisar e encontrar produtos pelo menu de navegacao','Q3',3), engine_OUT)
BASE_CONVIDADO_5 = pd.read_sql(processa_base("CONVIDADO",'NAVEGACAO','Facilidade em reconhecer as palavras na busca por um produto','Q4',3), engine_OUT)
BASE_CONVIDADO_6 = pd.read_sql(processa_base("CONVIDADO",'NAVEGACAO','Facilidade de comparar precos e produtos similares no site','Q5',3), engine_OUT)
BASE_CONVIDADO_7 = pd.read_sql(processa_base("CONVIDADO",'NAVEGACAO','Facilidade na conclusao da compra','Q6',3), engine_OUT)
BASE_CONVIDADO_8 = pd.read_sql(processa_base("CONVIDADO",'NAVEGACAO','Opcao de compra de uma Loja Parceira em nosso site','Q7',3), engine_OUT)
BASE_CONVIDADO_9 = pd.read_sql(processa_base("CONVIDADO",'NAVEGACAO','Seguranca em comprar de uma Loja Parceira','Q8',3), engine_OUT)
BASE_CONVIDADO_10 = pd.read_sql(processa_base("CONVIDADO",'NAVEGACAO','Facilidade para pesquisar e encontrar a Lista de Casamento','Q9',3), engine_OUT)
BASE_CONVIDADO_11 = pd.read_sql(processa_base("CONVIDADO",'NAVEGACAO','Facilidade em escolher o produto','Q10',3), engine_OUT)
BASE_CONVIDADO_12 = pd.read_sql(processa_base("CONVIDADO",'NAVEGACAO','Facilidade para escrever a mensagem aos noivos','Q11',3), engine_OUT)
BASE_CONVIDADO_13 = pd.read_sql(processa_base("CONVIDADO",'NAVEGACAO','Facilidade para concluir a compra','Q12',3), engine_OUT)
BASE_CONVIDADO_14 = pd.read_sql(processa_base("CONVIDADO",'NAVEGACAO','Facilidade para criar e editar a Lista de Casamento','Q13',3), engine_OUT)
BASE_CONVIDADO_15 = pd.read_sql(processa_base("CONVIDADO",'NAVEGACAO','Facilidade para divulgar a sua Lista de Casamento','Q14',3), engine_OUT)
BASE_CONVIDADO_16 = pd.read_sql(processa_base("CONVIDADO",'NAVEGACAO','Facilidade para converter os produtos em creditos ou libera-los para entrega','Q15',3), engine_OUT)
BASE_CONVIDADO_17 = pd.read_sql(processa_base("CONVIDADO",'NAVEGACAO','Facilidade para agradecer aos convidados pelos presentes ganhos','Q16',3), engine_OUT)
BASE_CONVIDADO_18 = pd.read_sql(processa_base("CONVIDADO",'NAVEGACAO','Facilidade para acompanhar o saldo acumulado dos creditos','Q17',3), engine_OUT)
BASE_CONVIDADO_19 = pd.read_sql(processa_base("CONVIDADO",'PRODUTOS','Variedade de produtos e marcas','Q18',3), engine_OUT)
BASE_CONVIDADO_20 = pd.read_sql(processa_base("CONVIDADO",'PRODUTOS','Facilidade de encontrar informacoes sobre o produto (descricao detalhada, fotos e avaliacao de outros compradores)','Q19',3), engine_OUT)
BASE_CONVIDADO_21 = pd.read_sql(processa_base("CONVIDADO",'PAGAMENTO','Variedade de opcoes de pagamento','Q20',3), engine_OUT)
BASE_CONVIDADO_22 = pd.read_sql(processa_base("CONVIDADO",'PAGAMENTO','Clareza da taxa de juros em caso de parcelamento','Q21',3), engine_OUT)
BASE_CONVIDADO_23 = pd.read_sql(processa_base("CONVIDADO",'PAGAMENTO','Tempo de aprovacao do pagamento','Q22',3), engine_OUT)
BASE_CONVIDADO_24 = pd.read_sql(processa_base("CONVIDADO",'PAGAMENTO','Quantidade de parcelas sem juros','Q23',3), engine_OUT)
BASE_CONVIDADO_25 = pd.read_sql(processa_base("CONVIDADO",'PAGAMENTO','Clareza do valor do Frete','Q24',3), engine_OUT)
BASE_CONVIDADO_26 = pd.read_sql(processa_base("CONVIDADO",'PAGAMENTO','Opcao de compra utilizando vale','Q25',3), engine_OUT)
BASE_CONVIDADO_27 = pd.read_sql(processa_base("CONVIDADO",'ENTREGA','Clareza do valor do frete','Q26',3), engine_OUT)
BASE_CONVIDADO_28 = pd.read_sql(processa_base("CONVIDADO",'ENTREGA','Opcao de retirar a compra na loja','Q27',3), engine_OUT)
BASE_CONVIDADO_29 = pd.read_sql(processa_base("CONVIDADO",'ENTREGA','Acompanhamento da entrega da compra','Q28',3), engine_OUT)
BASE_CONVIDADO_30 = pd.read_sql(processa_base("CONVIDADO",'ENTREGA','Entrega dentro do prazo estipulado','Q29',3), engine_OUT)
BASE_CONVIDADO_31 = pd.read_sql(processa_base("CONVIDADO",'ENTREGA','Condicoes do produto entregue','Q30',3), engine_OUT)
BASE_CONVIDADO_32 = pd.read_sql(processa_base("CONVIDADO",'COMPRA_LOJA_PARCEIRA_MKT','Clareza e transparencia que a compra sera realizada em uma Loja Parceira','Q31',3), engine_OUT)
BASE_CONVIDADO_33 = pd.read_sql(processa_base("CONVIDADO",'COMPRA_LOJA_PARCEIRA_MKT','Seguranca em comprar de uma Loja Parceira','Q32',3), engine_OUT)
BASE_CONVIDADO_34 = pd.read_sql(processa_base("CONVIDADO",'RETIRADA_LOJA_FISICA','Variedade de lojas disponiveis para retirada','Q33',3), engine_OUT)
BASE_CONVIDADO_35 = pd.read_sql(processa_base("CONVIDADO",'RETIRADA_LOJA_FISICA','Prazo atrativo para retirada da compra na loja fisica','Q34',3), engine_OUT)
BASE_CONVIDADO_36 = pd.read_sql(processa_base("CONVIDADO",'RETIRADA_LOJA_FISICA','Informacoes claras sobre a retirada da compra','Q35',3), engine_OUT)
BASE_CONVIDADO_37 = pd.read_sql(processa_base("CONVIDADO",'RETIRADA_LOJA_FISICA','Acompanhamento da disponibilidade de retirada da compra','Q36',3), engine_OUT)
BASE_CONVIDADO_38 = pd.read_sql(processa_base("CONVIDADO",'RETIRADA_LOJA_FISICA','Facilidade, cordialidade e tempo de espera durante o atendimento na retirada da compra','Q37',3), engine_OUT)
BASE_CONVIDADO_39 = pd.read_sql(processa_base("CONVIDADO",'RETIRADA_LOJA_FISICA','Retirada dentro do prazo estipulado','Q38',3), engine_OUT)
BASE_CONVIDADO_40 = pd.read_sql(processa_base("CONVIDADO",'ATENDIMENTO','Facilidade para entrar em contato com nossa Central de Atendimento','Q39',3), engine_OUT)
BASE_CONVIDADO_41 = pd.read_sql(processa_base("CONVIDADO",'ATENDIMENTO','Atencao, cordialidade e conhecimento dos atendentes','Q40',3), engine_OUT)
BASE_CONVIDADO_42 = pd.read_sql(processa_base("CONVIDADO",'ATENDIMENTO','Prazo informado para resolucao','Q41',3), engine_OUT)
print ('DONE CONVIDADO')

BASE_MARKETPLACE_1 = pd.read_sql(processa_base("MARKETPLACE",'NPS','Em uma escala de 0 a 10, o quanto voce recomendaria nossa empresa para um amigo?','Q0',4), engine_OUT)
BASE_MARKETPLACE_2 = pd.read_sql(processa_base("MARKETPLACE",'PRECOS_OFERTAS','Preco dos produtos em geral','Q1',4), engine_OUT)
BASE_MARKETPLACE_3 = pd.read_sql(processa_base("MARKETPLACE",'PRECOS_OFERTAS','Atratividade das ofertas em nosso site','Q2',4), engine_OUT)
BASE_MARKETPLACE_4 = pd.read_sql(processa_base("MARKETPLACE",'NAVEGACAO','Facilidade para pesquisar e encontrar produtos pelo menu de navegacao','Q3',4), engine_OUT)
BASE_MARKETPLACE_5 = pd.read_sql(processa_base("MARKETPLACE",'NAVEGACAO','Facilidade em reconhecer as palavras na busca por um produto','Q4',4), engine_OUT)
BASE_MARKETPLACE_6 = pd.read_sql(processa_base("MARKETPLACE",'NAVEGACAO','Facilidade de comparar precos e produtos similares no site','Q5',4), engine_OUT)
BASE_MARKETPLACE_7 = pd.read_sql(processa_base("MARKETPLACE",'NAVEGACAO','Facilidade na conclusao da compra','Q6',4), engine_OUT)
BASE_MARKETPLACE_8 = pd.read_sql(processa_base("MARKETPLACE",'NAVEGACAO','Opcao de compra de uma Loja Parceira em nosso site','Q7',4), engine_OUT)
BASE_MARKETPLACE_9 = pd.read_sql(processa_base("MARKETPLACE",'NAVEGACAO','Seguranca em comprar de uma Loja Parceira','Q8',4), engine_OUT)
BASE_MARKETPLACE_10 = pd.read_sql(processa_base("MARKETPLACE",'NAVEGACAO','Facilidade para pesquisar e encontrar a Lista de Casamento','Q9',4), engine_OUT)
BASE_MARKETPLACE_11 = pd.read_sql(processa_base("MARKETPLACE",'NAVEGACAO','Facilidade em escolher o produto','Q10',4), engine_OUT)
BASE_MARKETPLACE_12 = pd.read_sql(processa_base("MARKETPLACE",'NAVEGACAO','Facilidade para escrever a mensagem aos noivos','Q11',4), engine_OUT)
BASE_MARKETPLACE_13 = pd.read_sql(processa_base("MARKETPLACE",'NAVEGACAO','Facilidade para concluir a compra','Q12',4), engine_OUT)
BASE_MARKETPLACE_14 = pd.read_sql(processa_base("MARKETPLACE",'NAVEGACAO','Facilidade para criar e editar a Lista de Casamento','Q13',4), engine_OUT)
BASE_MARKETPLACE_15 = pd.read_sql(processa_base("MARKETPLACE",'NAVEGACAO','Facilidade para divulgar a sua Lista de Casamento','Q14',4), engine_OUT)
BASE_MARKETPLACE_16 = pd.read_sql(processa_base("MARKETPLACE",'NAVEGACAO','Facilidade para converter os produtos em creditos ou libera-los para entrega','Q15',4), engine_OUT)
BASE_MARKETPLACE_17 = pd.read_sql(processa_base("MARKETPLACE",'NAVEGACAO','Facilidade para agradecer aos convidados pelos presentes ganhos','Q16',4), engine_OUT)
BASE_MARKETPLACE_18 = pd.read_sql(processa_base("MARKETPLACE",'NAVEGACAO','Facilidade para acompanhar o saldo acumulado dos creditos','Q17',4), engine_OUT)
BASE_MARKETPLACE_19 = pd.read_sql(processa_base("MARKETPLACE",'PRODUTOS','Variedade de produtos e marcas','Q18',4), engine_OUT)
BASE_MARKETPLACE_20 = pd.read_sql(processa_base("MARKETPLACE",'PRODUTOS','Facilidade de encontrar informacoes sobre o produto (descricao detalhada, fotos e avaliacao de outros compradores)','Q19',4), engine_OUT)
BASE_MARKETPLACE_21 = pd.read_sql(processa_base("MARKETPLACE",'PAGAMENTO','Variedade de opcoes de pagamento','Q20',4), engine_OUT)
BASE_MARKETPLACE_22 = pd.read_sql(processa_base("MARKETPLACE",'PAGAMENTO','Clareza da taxa de juros em caso de parcelamento','Q21',4), engine_OUT)
BASE_MARKETPLACE_23 = pd.read_sql(processa_base("MARKETPLACE",'PAGAMENTO','Tempo de aprovacao do pagamento','Q22',4), engine_OUT)
BASE_MARKETPLACE_24 = pd.read_sql(processa_base("MARKETPLACE",'PAGAMENTO','Quantidade de parcelas sem juros','Q23',4), engine_OUT)
BASE_MARKETPLACE_25 = pd.read_sql(processa_base("MARKETPLACE",'PAGAMENTO','Clareza do valor do Frete','Q24',4), engine_OUT)
BASE_MARKETPLACE_26 = pd.read_sql(processa_base("MARKETPLACE",'PAGAMENTO','Opcao de compra utilizando vale','Q25',4), engine_OUT)
BASE_MARKETPLACE_27 = pd.read_sql(processa_base("MARKETPLACE",'ENTREGA','Clareza do valor do frete','Q26',4), engine_OUT)
BASE_MARKETPLACE_28 = pd.read_sql(processa_base("MARKETPLACE",'ENTREGA','Opcao de retirar a compra na loja','Q27',4), engine_OUT)
BASE_MARKETPLACE_29 = pd.read_sql(processa_base("MARKETPLACE",'ENTREGA','Acompanhamento da entrega da compra','Q28',4), engine_OUT)
BASE_MARKETPLACE_30 = pd.read_sql(processa_base("MARKETPLACE",'ENTREGA','Entrega dentro do prazo estipulado','Q29',4), engine_OUT)
BASE_MARKETPLACE_31 = pd.read_sql(processa_base("MARKETPLACE",'ENTREGA','Condicoes do produto entregue','Q30',4), engine_OUT)
BASE_MARKETPLACE_32 = pd.read_sql(processa_base("MARKETPLACE",'COMPRA_LOJA_PARCEIRA_MKT','Clareza e transparencia que a compra sera realizada em uma Loja Parceira','Q31',4), engine_OUT)
BASE_MARKETPLACE_33 = pd.read_sql(processa_base("MARKETPLACE",'COMPRA_LOJA_PARCEIRA_MKT','Seguranca em comprar de uma Loja Parceira','Q32',4), engine_OUT)
BASE_MARKETPLACE_34 = pd.read_sql(processa_base("MARKETPLACE",'RETIRADA_LOJA_FISICA','Variedade de lojas disponiveis para retirada','Q33',4), engine_OUT)
BASE_MARKETPLACE_35 = pd.read_sql(processa_base("MARKETPLACE",'RETIRADA_LOJA_FISICA','Prazo atrativo para retirada da compra na loja fisica','Q34',4), engine_OUT)
BASE_MARKETPLACE_36 = pd.read_sql(processa_base("MARKETPLACE",'RETIRADA_LOJA_FISICA','Informacoes claras sobre a retirada da compra','Q35',4), engine_OUT)
BASE_MARKETPLACE_37 = pd.read_sql(processa_base("MARKETPLACE",'RETIRADA_LOJA_FISICA','Acompanhamento da disponibilidade de retirada da compra','Q36',4), engine_OUT)
BASE_MARKETPLACE_38 = pd.read_sql(processa_base("MARKETPLACE",'RETIRADA_LOJA_FISICA','Facilidade, cordialidade e tempo de espera durante o atendimento na retirada da compra','Q37',4), engine_OUT)
BASE_MARKETPLACE_39 = pd.read_sql(processa_base("MARKETPLACE",'RETIRADA_LOJA_FISICA','Retirada dentro do prazo estipulado','Q38',4), engine_OUT)
BASE_MARKETPLACE_40 = pd.read_sql(processa_base("MARKETPLACE",'ATENDIMENTO','Facilidade para entrar em contato com nossa Central de Atendimento','Q39',4), engine_OUT)
BASE_MARKETPLACE_41 = pd.read_sql(processa_base("MARKETPLACE",'ATENDIMENTO','Atencao, cordialidade e conhecimento dos atendentes','Q40',4), engine_OUT)
BASE_MARKETPLACE_42 = pd.read_sql(processa_base("MARKETPLACE",'ATENDIMENTO','Prazo informado para resolucao','Q41',4), engine_OUT)
print('DONE MARKETPLACE')


BASE_MISTO_1 = pd.read_sql(processa_base("MISTO",'NPS','Em uma escala de 0 a 10, o quanto voce recomendaria nossa empresa para um amigo?','Q0',5), engine_OUT)
BASE_MISTO_2 = pd.read_sql(processa_base("MISTO",'PRECOS_OFERTAS','Preco dos produtos em geral','Q1',5), engine_OUT)
BASE_MISTO_3 = pd.read_sql(processa_base("MISTO",'PRECOS_OFERTAS','Atratividade das ofertas em nosso site','Q2',5), engine_OUT)
BASE_MISTO_4 = pd.read_sql(processa_base("MISTO",'NAVEGACAO','Facilidade para pesquisar e encontrar produtos pelo menu de navegacao','Q3',5), engine_OUT)
BASE_MISTO_5 = pd.read_sql(processa_base("MISTO",'NAVEGACAO','Facilidade em reconhecer as palavras na busca por um produto','Q4',5), engine_OUT)
BASE_MISTO_6 = pd.read_sql(processa_base("MISTO",'NAVEGACAO','Facilidade de comparar precos e produtos similares no site','Q5',5), engine_OUT)
BASE_MISTO_7 = pd.read_sql(processa_base("MISTO",'NAVEGACAO','Facilidade na conclusao da compra','Q6',5), engine_OUT)
BASE_MISTO_8 = pd.read_sql(processa_base("MISTO",'NAVEGACAO','Opcao de compra de uma Loja Parceira em nosso site','Q7',5), engine_OUT)
BASE_MISTO_9 = pd.read_sql(processa_base("MISTO",'NAVEGACAO','Seguranca em comprar de uma Loja Parceira','Q8',5), engine_OUT)
BASE_MISTO_10 = pd.read_sql(processa_base("MISTO",'NAVEGACAO','Facilidade para pesquisar e encontrar a Lista de Casamento','Q9',5), engine_OUT)
BASE_MISTO_11 = pd.read_sql(processa_base("MISTO",'NAVEGACAO','Facilidade em escolher o produto','Q10',5), engine_OUT)
BASE_MISTO_12 = pd.read_sql(processa_base("MISTO",'NAVEGACAO','Facilidade para escrever a mensagem aos noivos','Q11',5), engine_OUT)
BASE_MISTO_13 = pd.read_sql(processa_base("MISTO",'NAVEGACAO','Facilidade para concluir a compra','Q12',5), engine_OUT)
BASE_MISTO_14 = pd.read_sql(processa_base("MISTO",'NAVEGACAO','Facilidade para criar e editar a Lista de Casamento','Q13',5), engine_OUT)
BASE_MISTO_15 = pd.read_sql(processa_base("MISTO",'NAVEGACAO','Facilidade para divulgar a sua Lista de Casamento','Q14',5), engine_OUT)
BASE_MISTO_16 = pd.read_sql(processa_base("MISTO",'NAVEGACAO','Facilidade para converter os produtos em creditos ou libera-los para entrega','Q15',5), engine_OUT)
BASE_MISTO_17 = pd.read_sql(processa_base("MISTO",'NAVEGACAO','Facilidade para agradecer aos convidados pelos presentes ganhos','Q16',5), engine_OUT)
BASE_MISTO_18 = pd.read_sql(processa_base("MISTO",'NAVEGACAO','Facilidade para acompanhar o saldo acumulado dos creditos','Q17',5), engine_OUT)
BASE_MISTO_19 = pd.read_sql(processa_base("MISTO",'PRODUTOS','Variedade de produtos e marcas','Q18',5), engine_OUT)
BASE_MISTO_20 = pd.read_sql(processa_base("MISTO",'PRODUTOS','Facilidade de encontrar informacoes sobre o produto (descricao detalhada, fotos e avaliacao de outros compradores)','Q19',5), engine_OUT)
BASE_MISTO_21 = pd.read_sql(processa_base("MISTO",'PAGAMENTO','Variedade de opcoes de pagamento','Q20',5), engine_OUT)
BASE_MISTO_22 = pd.read_sql(processa_base("MISTO",'PAGAMENTO','Clareza da taxa de juros em caso de parcelamento','Q21',5), engine_OUT)
BASE_MISTO_23 = pd.read_sql(processa_base("MISTO",'PAGAMENTO','Tempo de aprovacao do pagamento','Q22',5), engine_OUT)
BASE_MISTO_24 = pd.read_sql(processa_base("MISTO",'PAGAMENTO','Quantidade de parcelas sem juros','Q23',5), engine_OUT)
BASE_MISTO_25 = pd.read_sql(processa_base("MISTO",'PAGAMENTO','Clareza do valor do Frete','Q24',5), engine_OUT)
BASE_MISTO_26 = pd.read_sql(processa_base("MISTO",'PAGAMENTO','Opcao de compra utilizando vale','Q25',5), engine_OUT)
BASE_MISTO_27 = pd.read_sql(processa_base("MISTO",'ENTREGA','Clareza do valor do frete','Q26',5), engine_OUT)
BASE_MISTO_28 = pd.read_sql(processa_base("MISTO",'ENTREGA','Opcao de retirar a compra na loja','Q27',5), engine_OUT)
BASE_MISTO_29 = pd.read_sql(processa_base("MISTO",'ENTREGA','Acompanhamento da entrega da compra','Q28',5), engine_OUT)
BASE_MISTO_30 = pd.read_sql(processa_base("MISTO",'ENTREGA','Entrega dentro do prazo estipulado','Q29',5), engine_OUT)
BASE_MISTO_31 = pd.read_sql(processa_base("MISTO",'ENTREGA','Condicoes do produto entregue','Q30',5), engine_OUT)
BASE_MISTO_32 = pd.read_sql(processa_base("MISTO",'COMPRA_LOJA_PARCEIRA_MKT','Clareza e transparencia que a compra sera realizada em uma Loja Parceira','Q31',5), engine_OUT)
BASE_MISTO_33 = pd.read_sql(processa_base("MISTO",'COMPRA_LOJA_PARCEIRA_MKT','Seguranca em comprar de uma Loja Parceira','Q32',5), engine_OUT)
BASE_MISTO_34 = pd.read_sql(processa_base("MISTO",'RETIRADA_LOJA_FISICA','Variedade de lojas disponiveis para retirada','Q33',5), engine_OUT)
BASE_MISTO_35 = pd.read_sql(processa_base("MISTO",'RETIRADA_LOJA_FISICA','Prazo atrativo para retirada da compra na loja fisica','Q34',5), engine_OUT)
BASE_MISTO_36 = pd.read_sql(processa_base("MISTO",'RETIRADA_LOJA_FISICA','Informacoes claras sobre a retirada da compra','Q35',5), engine_OUT)
BASE_MISTO_37 = pd.read_sql(processa_base("MISTO",'RETIRADA_LOJA_FISICA','Acompanhamento da disponibilidade de retirada da compra','Q36',5), engine_OUT)
BASE_MISTO_38 = pd.read_sql(processa_base("MISTO",'RETIRADA_LOJA_FISICA','Facilidade, cordialidade e tempo de espera durante o atendimento na retirada da compra','Q37',5), engine_OUT)
BASE_MISTO_39 = pd.read_sql(processa_base("MISTO",'RETIRADA_LOJA_FISICA','Retirada dentro do prazo estipulado','Q38',5), engine_OUT)
BASE_MISTO_40 = pd.read_sql(processa_base("MISTO",'ATENDIMENTO','Facilidade para entrar em contato com nossa Central de Atendimento','Q39',5), engine_OUT)
BASE_MISTO_41 = pd.read_sql(processa_base("MISTO",'ATENDIMENTO','Atencao, cordialidade e conhecimento dos atendentes','Q40',5), engine_OUT)
BASE_MISTO_42 = pd.read_sql(processa_base("MISTO",'ATENDIMENTO','Prazo informado para resolucao','Q41',5), engine_OUT)
print('DONE MISTO')

BASE_MULTICANALIDADE_1 = pd.read_sql(processa_base("MULTICANALIDADE",'NPS','Em uma escala de 0 a 10, o quanto voce recomendaria nossa empresa para um amigo?','Q0',6), engine_OUT)
BASE_MULTICANALIDADE_2 = pd.read_sql(processa_base("MULTICANALIDADE",'PRECOS_OFERTAS','Preco dos produtos em geral','Q1',6), engine_OUT)
BASE_MULTICANALIDADE_3 = pd.read_sql(processa_base("MULTICANALIDADE",'PRECOS_OFERTAS','Atratividade das ofertas em nosso site','Q2',6), engine_OUT)
BASE_MULTICANALIDADE_4 = pd.read_sql(processa_base("MULTICANALIDADE",'NAVEGACAO','Facilidade para pesquisar e encontrar produtos pelo menu de navegacao','Q3',6), engine_OUT)
BASE_MULTICANALIDADE_5 = pd.read_sql(processa_base("MULTICANALIDADE",'NAVEGACAO','Facilidade em reconhecer as palavras na busca por um produto','Q4',6), engine_OUT)
BASE_MULTICANALIDADE_6 = pd.read_sql(processa_base("MULTICANALIDADE",'NAVEGACAO','Facilidade de comparar precos e produtos similares no site','Q5',6), engine_OUT)
BASE_MULTICANALIDADE_7 = pd.read_sql(processa_base("MULTICANALIDADE",'NAVEGACAO','Facilidade na conclusao da compra','Q6',6), engine_OUT)
BASE_MULTICANALIDADE_8 = pd.read_sql(processa_base("MULTICANALIDADE",'NAVEGACAO','Opcao de compra de uma Loja Parceira em nosso site','Q7',6), engine_OUT)
BASE_MULTICANALIDADE_9 = pd.read_sql(processa_base("MULTICANALIDADE",'NAVEGACAO','Seguranca em comprar de uma Loja Parceira','Q8',6), engine_OUT)
BASE_MULTICANALIDADE_10 = pd.read_sql(processa_base("MULTICANALIDADE",'NAVEGACAO','Facilidade para pesquisar e encontrar a Lista de Casamento','Q9',6), engine_OUT)
BASE_MULTICANALIDADE_11 = pd.read_sql(processa_base("MULTICANALIDADE",'NAVEGACAO','Facilidade em escolher o produto','Q10',6), engine_OUT)
BASE_MULTICANALIDADE_12 = pd.read_sql(processa_base("MULTICANALIDADE",'NAVEGACAO','Facilidade para escrever a mensagem aos noivos','Q11',6), engine_OUT)
BASE_MULTICANALIDADE_13 = pd.read_sql(processa_base("MULTICANALIDADE",'NAVEGACAO','Facilidade para concluir a compra','Q12',6), engine_OUT)
BASE_MULTICANALIDADE_14 = pd.read_sql(processa_base("MULTICANALIDADE",'NAVEGACAO','Facilidade para criar e editar a Lista de Casamento','Q13',6), engine_OUT)
BASE_MULTICANALIDADE_15 = pd.read_sql(processa_base("MULTICANALIDADE",'NAVEGACAO','Facilidade para divulgar a sua Lista de Casamento','Q14',6), engine_OUT)
BASE_MULTICANALIDADE_16 = pd.read_sql(processa_base("MULTICANALIDADE",'NAVEGACAO','Facilidade para converter os produtos em creditos ou libera-los para entrega','Q15',6), engine_OUT)
BASE_MULTICANALIDADE_17 = pd.read_sql(processa_base("MULTICANALIDADE",'NAVEGACAO','Facilidade para agradecer aos convidados pelos presentes ganhos','Q16',6), engine_OUT)
BASE_MULTICANALIDADE_18 = pd.read_sql(processa_base("MULTICANALIDADE",'NAVEGACAO','Facilidade para acompanhar o saldo acumulado dos creditos','Q17',6), engine_OUT)
BASE_MULTICANALIDADE_19 = pd.read_sql(processa_base("MULTICANALIDADE",'PRODUTOS','Variedade de produtos e marcas','Q18',6), engine_OUT)
BASE_MULTICANALIDADE_20 = pd.read_sql(processa_base("MULTICANALIDADE",'PRODUTOS','Facilidade de encontrar informacoes sobre o produto (descricao detalhada, fotos e avaliacao de outros compradores)','Q19',6), engine_OUT)
BASE_MULTICANALIDADE_21 = pd.read_sql(processa_base("MULTICANALIDADE",'PAGAMENTO','Variedade de opcoes de pagamento','Q20',6), engine_OUT)
BASE_MULTICANALIDADE_22 = pd.read_sql(processa_base("MULTICANALIDADE",'PAGAMENTO','Clareza da taxa de juros em caso de parcelamento','Q21',6), engine_OUT)
BASE_MULTICANALIDADE_23 = pd.read_sql(processa_base("MULTICANALIDADE",'PAGAMENTO','Tempo de aprovacao do pagamento','Q22',6), engine_OUT)
BASE_MULTICANALIDADE_24 = pd.read_sql(processa_base("MULTICANALIDADE",'PAGAMENTO','Quantidade de parcelas sem juros','Q23',6), engine_OUT)
BASE_MULTICANALIDADE_25 = pd.read_sql(processa_base("MULTICANALIDADE",'PAGAMENTO','Clareza do valor do Frete','Q24',6), engine_OUT)
BASE_MULTICANALIDADE_26 = pd.read_sql(processa_base("MULTICANALIDADE",'PAGAMENTO','Opcao de compra utilizando vale','Q25',6), engine_OUT)
BASE_MULTICANALIDADE_27 = pd.read_sql(processa_base("MULTICANALIDADE",'ENTREGA','Clareza do valor do frete','Q26',6), engine_OUT)
BASE_MULTICANALIDADE_28 = pd.read_sql(processa_base("MULTICANALIDADE",'ENTREGA','Opcao de retirar a compra na loja','Q27',6), engine_OUT)
BASE_MULTICANALIDADE_29 = pd.read_sql(processa_base("MULTICANALIDADE",'ENTREGA','Acompanhamento da entrega da compra','Q28',6), engine_OUT)
BASE_MULTICANALIDADE_30 = pd.read_sql(processa_base("MULTICANALIDADE",'ENTREGA','Entrega dentro do prazo estipulado','Q29',6), engine_OUT)
BASE_MULTICANALIDADE_31 = pd.read_sql(processa_base("MULTICANALIDADE",'ENTREGA','Condicoes do produto entregue','Q30',6), engine_OUT)
BASE_MULTICANALIDADE_32 = pd.read_sql(processa_base("MULTICANALIDADE",'COMPRA_LOJA_PARCEIRA_MKT','Clareza e transparencia que a compra sera realizada em uma Loja Parceira','Q31',6), engine_OUT)
BASE_MULTICANALIDADE_33 = pd.read_sql(processa_base("MULTICANALIDADE",'COMPRA_LOJA_PARCEIRA_MKT','Seguranca em comprar de uma Loja Parceira','Q32',6), engine_OUT)
BASE_MULTICANALIDADE_34 = pd.read_sql(processa_base("MULTICANALIDADE",'RETIRADA_LOJA_FISICA','Variedade de lojas disponiveis para retirada','Q33',6), engine_OUT)
BASE_MULTICANALIDADE_35 = pd.read_sql(processa_base("MULTICANALIDADE",'RETIRADA_LOJA_FISICA','Prazo atrativo para retirada da compra na loja fisica','Q34',6), engine_OUT)
BASE_MULTICANALIDADE_36 = pd.read_sql(processa_base("MULTICANALIDADE",'RETIRADA_LOJA_FISICA','Informacoes claras sobre a retirada da compra','Q35',6), engine_OUT)
BASE_MULTICANALIDADE_37 = pd.read_sql(processa_base("MULTICANALIDADE",'RETIRADA_LOJA_FISICA','Acompanhamento da disponibilidade de retirada da compra','Q36',6), engine_OUT)
BASE_MULTICANALIDADE_38 = pd.read_sql(processa_base("MULTICANALIDADE",'RETIRADA_LOJA_FISICA','Facilidade, cordialidade e tempo de espera durante o atendimento na retirada da compra','Q37',6), engine_OUT)
BASE_MULTICANALIDADE_39 = pd.read_sql(processa_base("MULTICANALIDADE",'RETIRADA_LOJA_FISICA','Retirada dentro do prazo estipulado','Q38',6), engine_OUT)
BASE_MULTICANALIDADE_40 = pd.read_sql(processa_base("MULTICANALIDADE",'ATENDIMENTO','Facilidade para entrar em contato com nossa Central de Atendimento','Q39',6), engine_OUT)
BASE_MULTICANALIDADE_41 = pd.read_sql(processa_base("MULTICANALIDADE",'ATENDIMENTO','Atencao, cordialidade e conhecimento dos atendentes','Q40',6), engine_OUT)
BASE_MULTICANALIDADE_42 = pd.read_sql(processa_base("MULTICANALIDADE",'ATENDIMENTO','Prazo informado para resolucao','Q41',6), engine_OUT)
print('DONE MULTICANALIDADE')

print('FIM DA SEPARACAO DOS QUESTIONARIOS')

B2C= [BASE_B2C_1 ,BASE_B2C_2,	BASE_B2C_3,	BASE_B2C_4,	BASE_B2C_5,	BASE_B2C_6,	BASE_B2C_7,	BASE_B2C_8,	BASE_B2C_9,	BASE_B2C_10,	BASE_B2C_11,	BASE_B2C_12,	BASE_B2C_13,	BASE_B2C_14,	BASE_B2C_15,	BASE_B2C_16,	BASE_B2C_17,	BASE_B2C_18,	BASE_B2C_19,	BASE_B2C_20,	BASE_B2C_21,	BASE_B2C_22,	BASE_B2C_23,	BASE_B2C_24,	BASE_B2C_25,	BASE_B2C_26,	BASE_B2C_27,	BASE_B2C_28,	BASE_B2C_29,	BASE_B2C_30,	BASE_B2C_31,	BASE_B2C_32,	BASE_B2C_33,	BASE_B2C_34,	BASE_B2C_35,	BASE_B2C_36,	BASE_B2C_37,	BASE_B2C_38,	BASE_B2C_39,	BASE_B2C_40,	BASE_B2C_41,	BASE_B2C_42]
B2C=pd.concat(B2C)
print('DONE CONCATENATING B2C')

NOIVO= [BASE_NOIVO_1 ,	BASE_NOIVO_2,	BASE_NOIVO_3,	BASE_NOIVO_4,	BASE_NOIVO_5,	BASE_NOIVO_6,	BASE_NOIVO_7,	BASE_NOIVO_8,	BASE_NOIVO_9,	BASE_NOIVO_10,	BASE_NOIVO_11,	BASE_NOIVO_12,	BASE_NOIVO_13,	BASE_NOIVO_14,	BASE_NOIVO_15,	BASE_NOIVO_16,	BASE_NOIVO_17,	BASE_NOIVO_18,	BASE_NOIVO_19,	BASE_NOIVO_20,	BASE_NOIVO_21,	BASE_NOIVO_22,	BASE_NOIVO_23,	BASE_NOIVO_24,	BASE_NOIVO_25,	BASE_NOIVO_26,	BASE_NOIVO_27,	BASE_NOIVO_28,	BASE_NOIVO_29,	BASE_NOIVO_30,	BASE_NOIVO_31,	BASE_NOIVO_32,	BASE_NOIVO_33,	BASE_NOIVO_34,	BASE_NOIVO_35,	BASE_NOIVO_36,	BASE_NOIVO_37,	BASE_NOIVO_38,	BASE_NOIVO_39,	BASE_NOIVO_40,	BASE_NOIVO_41,	BASE_NOIVO_42]
NOIVO=pd.concat(NOIVO)
print('DONE CONCATENATING NOIVO')

CONVIDADO = [BASE_CONVIDADO_1 ,	BASE_CONVIDADO_2,	BASE_CONVIDADO_3,	BASE_CONVIDADO_4,	BASE_CONVIDADO_5,	BASE_CONVIDADO_6,	BASE_CONVIDADO_7,	BASE_CONVIDADO_8,	BASE_CONVIDADO_9,	BASE_CONVIDADO_10,	BASE_CONVIDADO_11,	BASE_CONVIDADO_12,	BASE_CONVIDADO_13,	BASE_CONVIDADO_14,	BASE_CONVIDADO_15,	BASE_CONVIDADO_16,	BASE_CONVIDADO_17,	BASE_CONVIDADO_18,	BASE_CONVIDADO_19,	BASE_CONVIDADO_20,	BASE_CONVIDADO_21,	BASE_CONVIDADO_22,	BASE_CONVIDADO_23,	BASE_CONVIDADO_24,	BASE_CONVIDADO_25,	BASE_CONVIDADO_26,	BASE_CONVIDADO_27,	BASE_CONVIDADO_28,	BASE_CONVIDADO_29,	BASE_CONVIDADO_30,	BASE_CONVIDADO_31,	BASE_CONVIDADO_32,	BASE_CONVIDADO_33,	BASE_CONVIDADO_34,	BASE_CONVIDADO_35,	BASE_CONVIDADO_36,	BASE_CONVIDADO_37,	BASE_CONVIDADO_38,	BASE_CONVIDADO_39,	BASE_CONVIDADO_40,	BASE_CONVIDADO_41,	BASE_CONVIDADO_42]
CONVIDADO = pd.concat(CONVIDADO)
print('DONE CONVIDADO')

MARKETPLACE = [BASE_MARKETPLACE_1 ,	BASE_MARKETPLACE_2,	BASE_MARKETPLACE_3,	BASE_MARKETPLACE_4,	BASE_MARKETPLACE_5,	BASE_MARKETPLACE_6,	BASE_MARKETPLACE_7,	BASE_MARKETPLACE_8,	BASE_MARKETPLACE_9,	BASE_MARKETPLACE_10,	BASE_MARKETPLACE_11,	BASE_MARKETPLACE_12,	BASE_MARKETPLACE_13,	BASE_MARKETPLACE_14,	BASE_MARKETPLACE_15,	BASE_MARKETPLACE_16,	BASE_MARKETPLACE_17,	BASE_MARKETPLACE_18,	BASE_MARKETPLACE_19,	BASE_MARKETPLACE_20,	BASE_MARKETPLACE_21,	BASE_MARKETPLACE_22,	BASE_MARKETPLACE_23,	BASE_MARKETPLACE_24,	BASE_MARKETPLACE_25,	BASE_MARKETPLACE_26,	BASE_MARKETPLACE_27,	BASE_MARKETPLACE_28,	BASE_MARKETPLACE_29,	BASE_MARKETPLACE_30,	BASE_MARKETPLACE_31,	BASE_MARKETPLACE_32,	BASE_MARKETPLACE_33,	BASE_MARKETPLACE_34,	BASE_MARKETPLACE_35,	BASE_MARKETPLACE_36,	BASE_MARKETPLACE_37,	BASE_MARKETPLACE_38,	BASE_MARKETPLACE_39,	BASE_MARKETPLACE_40,	BASE_MARKETPLACE_41,	BASE_MARKETPLACE_42]
MARKETPLACE = pd.concat(MARKETPLACE)
print('DONE MARKETPLACE')

MISTO = [BASE_MISTO_1 ,	BASE_MISTO_2,	BASE_MISTO_3,	BASE_MISTO_4,	BASE_MISTO_5,	BASE_MISTO_6,	BASE_MISTO_7,	BASE_MISTO_8,	BASE_MISTO_9,	BASE_MISTO_10,	BASE_MISTO_11,	BASE_MISTO_12,	BASE_MISTO_13,	BASE_MISTO_14,	BASE_MISTO_15,	BASE_MISTO_16,	BASE_MISTO_17,	BASE_MISTO_18,	BASE_MISTO_19,	BASE_MISTO_20,	BASE_MISTO_21,	BASE_MISTO_22,	BASE_MISTO_23,	BASE_MISTO_24,	BASE_MISTO_25,	BASE_MISTO_26,	BASE_MISTO_27,	BASE_MISTO_28,	BASE_MISTO_29,	BASE_MISTO_30,	BASE_MISTO_31,	BASE_MISTO_32,	BASE_MISTO_33,	BASE_MISTO_34,	BASE_MISTO_35,	BASE_MISTO_36,	BASE_MISTO_37,	BASE_MISTO_38,	BASE_MISTO_39,	BASE_MISTO_40,	BASE_MISTO_41,	BASE_MISTO_42]
MISTO = pd.concat(MISTO)
print('DONE MISTO')

MULTICANALIDADE = [BASE_MULTICANALIDADE_1 ,	BASE_MULTICANALIDADE_2,	BASE_MULTICANALIDADE_3,	BASE_MULTICANALIDADE_4,	BASE_MULTICANALIDADE_5,	BASE_MULTICANALIDADE_6,	BASE_MULTICANALIDADE_7,	BASE_MULTICANALIDADE_8,	BASE_MULTICANALIDADE_9,	BASE_MULTICANALIDADE_10,	BASE_MULTICANALIDADE_11,	BASE_MULTICANALIDADE_12,	BASE_MULTICANALIDADE_13,	BASE_MULTICANALIDADE_14,	BASE_MULTICANALIDADE_15,	BASE_MULTICANALIDADE_16,	BASE_MULTICANALIDADE_17,	BASE_MULTICANALIDADE_18,	BASE_MULTICANALIDADE_19,	BASE_MULTICANALIDADE_20,	BASE_MULTICANALIDADE_21,	BASE_MULTICANALIDADE_22,	BASE_MULTICANALIDADE_23,	BASE_MULTICANALIDADE_24,	BASE_MULTICANALIDADE_25,	BASE_MULTICANALIDADE_26,	BASE_MULTICANALIDADE_27,	BASE_MULTICANALIDADE_28,	BASE_MULTICANALIDADE_29,	BASE_MULTICANALIDADE_30,	BASE_MULTICANALIDADE_31,	BASE_MULTICANALIDADE_32,	BASE_MULTICANALIDADE_33,	BASE_MULTICANALIDADE_34,	BASE_MULTICANALIDADE_35,	BASE_MULTICANALIDADE_36,	BASE_MULTICANALIDADE_37,	BASE_MULTICANALIDADE_38,	BASE_MULTICANALIDADE_39,	BASE_MULTICANALIDADE_40,	BASE_MULTICANALIDADE_41,	BASE_MULTICANALIDADE_42]
MULTICANALIDADE = pd.concat(MULTICANALIDADE)
print('DONE MULTICANALIDADE')

VV_BASE_1 = [B2C,NOIVO,CONVIDADO,MARKETPLACE,MISTO,MULTICANALIDADE]
VV_BASE_1 = pd.concat(VV_BASE_1)
print('DONE ALL')

DONE B2C
DONE NOIVO
DONE CONVIDADO
DONE MARKETPLACE
DONE MISTO
DONE MULTICANALIDADE
FIM DA SEPARACAO DOS QUESTIONARIOS
DONE CONCATENATING B2C
DONE CONCATENATING NOIVO
DONE CONVIDADO
DONE MARKETPLACE
DONE MISTO
DONE MULTICANALIDADE
DONE ALL


# Appending Historic Information

In [9]:
BASE_LY['ID_QUESTIONARIO'] = BASE_LY['ID_SEGMENTO']
VV_BASE_1.columns = [x.upper() for x in VV_BASE_1.columns]
BASE_LY.columns = [x.upper() for x in BASE_LY.columns]
VV_BASE_2 = VV_BASE_1.append(BASE_LY)
VV_BASE_2

,ANO,ATRIBUTO,BANDEIRA,BANDEIRA_NUM_PEDIDO,DATA_ENTREVISTA,DATE_ID,DEPTO,DIMENSAO,DS_TIPO_ENTREGA,FLAG_PERIODO,...,METODOLOGIA,NOME_SEGMENTO,NR_LOJISTA,QUESTIONARIO,RESPID,SETOR,SEXO,TIPO_ENTREGA,UF,VALOR
0,2018.0,"Em uma escala de 0 a 10, o quanto voce recomen...",CB,136827912CB,01/18,16/01/2018,Eletroport?teis,NPS,Normal,CY,...,NaN,B2C,NaN,B2C,107679,Liquidificadores,F,CLIENTE,SP,10
1,2018.0,"Em uma escala de 0 a 10, o quanto voce recomen...",EX,137014888EX,01/18,24/01/2018,Brinquedos,NPS,Normal,CY,...,NaN,B2C,NaN,B2C,132293,Bonecos,M,CLIENTE,SP,10
2,NaN,"Em uma escala de 0 a 10, o quanto voce recomen...",EX,129932020EX,/,0,0,NPS,0,CY,...,NaN,B2C,NaN,B2C,2239,0,None,CLIENTE,0,10
3,NaN,"Em uma escala de 0 a 10, o quanto voce recomen...",CB,130931636CB,/,0,0,NPS,0,CY,...,NaN,B2C,NaN,B2C,3573,0,None,CLIENTE,0,9
4,2018.0,"Em uma escala de 0 a 10, o quanto voce recomen...",CB,137630117CB,01/18,27/01/2018,Eletrodom?sticos,NPS,Normal,CY,...,NaN,B2C,NaN,B2C,140905,Refrigeradores,F,CLIENTE,PR,8
5,2018.0,"Em uma escala de 0 a 10, o quanto voce recomen...",EX,135725324EX,01/18,19/01/2018,Telefones e Celulares,NPS,Normal,CY,...,NaN,B2C,NaN,B2C,121206,Smartphones Desbloqueados,M,CLIENTE,SE,10
6,2018.0,"Em uma escala de 0 a 10, o quanto voce recomen...",EX,136480707EX,01/18,18/01/2018,Telefones e Celulares,NPS,Normal,CY,...,NaN,B2C,NaN,B2C,118936,Smartphones Desbloqueados,M,CLIENTE,BA,8
7,2018.0,"Em uma escala de 0 a 10, o quanto voce recomen...",CB,137063734CB,01/18,18/01/2018,M??veis,NPS,Normal,CY,...,NaN,B2C,NaN,B2C,116289,Cozinha,M,CLIENTE,SP,4
8,2018.0,"Em uma escala de 0 a 10, o quanto voce recomen...",CB,136986293CB,01/18,29/01/2018,Eletrodom?sticos,NPS,Normal,CY,...,NaN,B2C,NaN,B2C,147698,Lavadoras,F,CLIENTE,BA,10
9,2018.0,"Em uma escala de 0 a 10, o quanto voce recomen...",CB,137670521CB,01/18,29/01/2018,Eletroport?teis,NPS,Normal,CY,...,NaN,B2C,NaN,B2C,147090,Aspirador de P?,M,CLIENTE,SC,7


<b><h1>ADDING RANKS PER ATRIBUTES</b></h1>

In [10]:
VV_BASE_FULL = pd.merge(VV_BASE_2, RANKS, how='left', left_on=['BANDEIRA','QUESTIONARIO','DIMENSAO','ATRIBUTO'],right_on=['BANDEIRA','SEGMENTO','DIMENSAO','ATRIBUTO'])
VV_BASE_FULL

,ANO,ATRIBUTO,BANDEIRA,BANDEIRA_NUM_PEDIDO,DATA_ENTREVISTA,DATE_ID,DEPTO,DIMENSAO,DS_TIPO_ENTREGA,FLAG_PERIODO,...,NR_LOJISTA,QUESTIONARIO,RESPID,SETOR,SEXO,TIPO_ENTREGA,UF,VALOR,RANK,SEGMENTO
0,2018.0,"Em uma escala de 0 a 10, o quanto voce recomen...",CB,136827912CB,01/18,16/01/2018,Eletroport?teis,NPS,Normal,CY,...,NaN,B2C,107679,Liquidificadores,F,CLIENTE,SP,10,NaN,NaN
1,2018.0,"Em uma escala de 0 a 10, o quanto voce recomen...",EX,137014888EX,01/18,24/01/2018,Brinquedos,NPS,Normal,CY,...,NaN,B2C,132293,Bonecos,M,CLIENTE,SP,10,NaN,NaN
2,NaN,"Em uma escala de 0 a 10, o quanto voce recomen...",EX,129932020EX,/,0,0,NPS,0,CY,...,NaN,B2C,2239,0,None,CLIENTE,0,10,NaN,NaN
3,NaN,"Em uma escala de 0 a 10, o quanto voce recomen...",CB,130931636CB,/,0,0,NPS,0,CY,...,NaN,B2C,3573,0,None,CLIENTE,0,9,NaN,NaN
4,2018.0,"Em uma escala de 0 a 10, o quanto voce recomen...",CB,137630117CB,01/18,27/01/2018,Eletrodom?sticos,NPS,Normal,CY,...,NaN,B2C,140905,Refrigeradores,F,CLIENTE,PR,8,NaN,NaN
5,2018.0,"Em uma escala de 0 a 10, o quanto voce recomen...",EX,135725324EX,01/18,19/01/2018,Telefones e Celulares,NPS,Normal,CY,...,NaN,B2C,121206,Smartphones Desbloqueados,M,CLIENTE,SE,10,NaN,NaN
6,2018.0,"Em uma escala de 0 a 10, o quanto voce recomen...",EX,136480707EX,01/18,18/01/2018,Telefones e Celulares,NPS,Normal,CY,...,NaN,B2C,118936,Smartphones Desbloqueados,M,CLIENTE,BA,8,NaN,NaN
7,2018.0,"Em uma escala de 0 a 10, o quanto voce recomen...",CB,137063734CB,01/18,18/01/2018,M??veis,NPS,Normal,CY,...,NaN,B2C,116289,Cozinha,M,CLIENTE,SP,4,NaN,NaN
8,2018.0,"Em uma escala de 0 a 10, o quanto voce recomen...",CB,136986293CB,01/18,29/01/2018,Eletrodom?sticos,NPS,Normal,CY,...,NaN,B2C,147698,Lavadoras,F,CLIENTE,BA,10,NaN,NaN
9,2018.0,"Em uma escala de 0 a 10, o quanto voce recomen...",CB,137670521CB,01/18,29/01/2018,Eletroport?teis,NPS,Normal,CY,...,NaN,B2C,147090,Aspirador de P?,M,CLIENTE,SC,7,NaN,NaN


<b><h1>UPLOADING TO EXADATA</b></h1>

In [ ]:
dtype2={
'RANK' : sa.NVARCHAR(length=255) ,
'questionario' : sa.NVARCHAR(length=255) ,
'dimensao' : sa.NVARCHAR(length=255) ,
'atributo' : sa.NVARCHAR(length=255) ,
'id_questionario' : sa.FLOAT ,
'respstatus' : sa.FLOAT ,
'capture_link' : sa.FLOAT ,
'id_bandeira' : sa.FLOAT ,
'id_segmento' : sa.FLOAT ,
'id_entrega' : sa.FLOAT ,
'tipo_lista' : sa.NVARCHAR(length=255) ,
'question_quota' : sa.FLOAT ,
'bandeira' : sa.NVARCHAR(length=255) ,
'flag_periodo' : sa.NVARCHAR(length=255) ,
'num_pedido' : sa.FLOAT ,
'bandeira_num_pedido' : sa.NVARCHAR(length=255) ,
'nome_segmento' : sa.NVARCHAR(length=255) ,
'tipo_entrega' : sa.NVARCHAR(length=255) ,
'date_id' : sa.NVARCHAR(length=255) ,
'data_entrevista' : sa.NVARCHAR(length=255) ,
'ano' : sa.NVARCHAR(length=255), 
'mes' : sa.NVARCHAR(length=255),
'mesano' : sa.NVARCHAR(length=255),
'num_cliente' : sa.FLOAT ,
'id_cliente' : sa.FLOAT ,
'ent' : sa.NVARCHAR(length=255) ,
'data_pedido' : sa.NVARCHAR(length=255) ,
'n_entrega' : sa.FLOAT ,
'num_lojista' : sa.FLOAT ,
'lojista' : sa.NVARCHAR(length=255) ,
'ponto_controle': sa.NVARCHAR(length=255),
'data_ponto' : sa.NVARCHAR(length=255), 
'sexo' : sa.NVARCHAR(length=255), 
'idade' : sa.INTEGER, 
'uf' : sa.NVARCHAR(length=255), 
'depto' : sa.NVARCHAR(length=255), 
'setor' : sa.NVARCHAR(length=255), 
'valor_pedido' : sa.FLOAT,
'qtd_produtos' :sa.FLOAT,
'valor' : sa.FLOAT,
'isc' : sa.FLOAT}

VV_BASE_FULL = VV_BASE_FULL.fillna(0)

# df1 = df.replace(np.nan, '', regex=True)

# VV_BASE_FULL['BANDEIRA'] = VV_BASE_FULL['BANDEIRA_x']

# del(VV_BASE_FULL['BANDEIRA_y'])
# del(VV_BASE_FULL['DIMENSAO'])
# del(VV_BASE_FULL['ATRIBUTO'])
# del(VV_BASE_FULL['ID_SEGMENTO'])
# VV_BASE_FULL['valor_pedido']=VV_BASE_FULL['valor_pedido'].apply(lambda x: x.replace(',','.'))
VV_BASE_FULL['VALOR'] =(VV_BASE_FULL['VALOR']).astype('float')
VV_BASE_FULL['IDADE'] =(VV_BASE_FULL['IDADE']).astype('str')

for column_name, column in VV_BASE_FULL.transpose().iterrows():
    print column_name
    print(VV_BASE_FULL[column_name])
    VV_BASE_FULL[column_name] = VV_BASE_FULL[column_name].astype('str')
    

VV_BASE_FULL.to_sql('vv_base_dash',engine_OUT,schema=None,if_exists='replace', chunksize=500, index=False,dtype={col_name: NVARCHAR(255) for col_name in VV_BASE_FULL})

VV_BASE_FULL

ANO
0          2018.0
1          2018.0
2             0.0
3             0.0
4          2018.0
5          2018.0
6          2018.0
7          2018.0
8          2018.0
9          2018.0
10         2018.0
11         2018.0
12         2018.0
13         2018.0
14         2018.0
15         2018.0
16         2018.0
17            0.0
18            0.0
19            0.0
20            0.0
21            0.0
22            0.0
23            0.0
24            0.0
25            0.0
26            0.0
27            0.0
28            0.0
29            0.0
            ...  
3758135    2017.0
3758136    2017.0
3758137    2017.0
3758138    2017.0
3758139    2017.0
3758140    2017.0
3758141    2017.0
3758142    2017.0
3758143    2017.0
3758144    2017.0
3758145    2017.0
3758146    2017.0
3758147    2017.0
3758148    2017.0
3758149    2017.0
3758150    2017.0
3758151    2017.0
3758152    2017.0
3758153    2017.0
3758154    2017.0
3758155    2017.0
3758156    2017.0
3758157    2017.0
3758158    2017.0
375815

DATE_ID
0          16/01/2018
1          24/01/2018
2                   0
3                   0
4          27/01/2018
5          19/01/2018
6          18/01/2018
7          18/01/2018
8          29/01/2018
9          29/01/2018
10         18/01/2018
11         28/01/2018
12         28/01/2018
13         25/01/2018
14         16/01/2018
15         16/01/2018
16         27/01/2018
17                  0
18                  0
19                  0
20                  0
21                  0
22                  0
23                  0
24                  0
25                  0
26                  0
27                  0
28                  0
29                  0
              ...    
3758135    01/11/2017
3758136    01/04/2017
3758137    01/01/2017
3758138    01/01/2017
3758139    01/11/2017
3758140    01/11/2017
3758141    01/11/2017
3758142    01/01/2017
3758143    01/01/2017
3758144    01/01/2017
3758145    01/01/2017
3758146    01/11/2017
3758147    01/04/2017
3758148    01/01/2017
37

ID_COMPRA
0                  0
1                  0
2                  0
3                  0
4                  0
5                  0
6                  0
7                  0
8                  0
9                  0
10                 0
11                 0
12                 0
13                 0
14                 0
15                 0
16                 0
17                 0
18                 0
19                 0
20                 0
21                 0
22                 0
23                 0
24                 0
25                 0
26                 0
27                 0
28                 0
29                 0
             ...    
3758135    106942508
3758136    106942306
3758137    106942235
3758138    106942235
3758139    106942480
3758140    106942506
3758141    106942452
3758142    106942198
3758143    106942198
3758144    106942240
3758145    106942240
3758146    106942460
3758147    106942274
3758148    106942199
3758149    106942199
3758150    106942328
375

QUESTIONARIO
0          B2C
1          B2C
2          B2C
3          B2C
4          B2C
5          B2C
6          B2C
7          B2C
8          B2C
9          B2C
10         B2C
11         B2C
12         B2C
13         B2C
14         B2C
15         B2C
16         B2C
17         B2C
18         B2C
19         B2C
20         B2C
21         B2C
22         B2C
23         B2C
24         B2C
25         B2C
26         B2C
27         B2C
28         B2C
29         B2C
          ... 
3758135    B2C
3758136    B2C
3758137    B2C
3758138    B2C
3758139    B2C
3758140    B2C
3758141    B2C
3758142    B2C
3758143    B2C
3758144    B2C
3758145    B2C
3758146    B2C
3758147    B2C
3758148    B2C
3758149    B2C
3758150    B2C
3758151    B2C
3758152    B2C
3758153    B2C
3758154    B2C
3758155    B2C
3758156    B2C
3758157    B2C
3758158    B2C
3758159    B2C
3758160    B2C
3758161    B2C
3758162    B2C
3758163    B2C
3758164    B2C
Name: QUESTIONARIO, Length: 3758165, dtype: object
RESPID
0          107

RANK
0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
5          0.0
6          0.0
7          0.0
8          0.0
9          0.0
10         0.0
11         0.0
12         0.0
13         0.0
14         0.0
15         0.0
16         0.0
17         0.0
18         0.0
19         0.0
20         0.0
21         0.0
22         0.0
23         0.0
24         0.0
25         0.0
26         0.0
27         0.0
28         0.0
29         0.0
          ... 
3758135    0.0
3758136    0.0
3758137    0.0
3758138    0.0
3758139    0.0
3758140    0.0
3758141    0.0
3758142    0.0
3758143    0.0
3758144    0.0
3758145    0.0
3758146    0.0
3758147    0.0
3758148    0.0
3758149    0.0
3758150    0.0
3758151    0.0
3758152    0.0
3758153    0.0
3758154    0.0
3758155    0.0
3758156    0.0
3758157    0.0
3758158    0.0
3758159    0.0
3758160    0.0
3758161    0.0
3758162    0.0
3758163    0.0
3758164    0.0
Name: RANK, Length: 3758165, dtype: object
SEGMENTO
0          0
1          0
2 

/usr/lib64/python2.7/site-packages/sqlalchemy/sql/sqltypes.py:226: SAWarning: Unicode type received non-unicode bind param value 'B2C'. (this warning may be suppressed after 10 occurrences)
  (util.ellipses_string(value),))
/usr/lib64/python2.7/site-packages/sqlalchemy/sql/sqltypes.py:226: SAWarning: Unicode type received non-unicode bind param value '2018.0'. (this warning may be suppressed after 10 occurrences)
  (util.ellipses_string(value),))
/usr/lib64/python2.7/site-packages/sqlalchemy/sql/sqltypes.py:226: SAWarning: Unicode type received non-unicode bind param value '0.0'. (this warning may be suppressed after 10 occurrences)
  (util.ellipses_string(value),))
/usr/lib64/python2.7/site-packages/sqlalchemy/sql/sqltypes.py:226: SAWarning: Unicode type received non-unicode bind param value '1'. (this warning may be suppressed after 10 occurrences)
  (util.ellipses_string(value),))
/usr/lib64/python2.7/site-packages/sqlalchemy/sql/sqltypes.py:226: SAWarning: Unicode type received non

<b><h1>CALCULATING NPS, PROMOTORES AND DETRATORES</b></h1>

In [14]:
COUNT_TOTAL="""
SELECT

RESPID,
QUESTIONARIO,
FLAG_PERIODO,
BANDEIRA,
DATA_ENTREVISTA,
DATE_ID,
ANO,
NOME_SEGMENTO,
SEXO,
IDADE,
UF,
DEPTO,
SETOR,
DS_TIPO_ENTREGA,
METODOLOGIA,
COUNT(VALOR) AS TOTAL

FROM vv_base_dash

WHERE DIMENSAO = 'NPS' AND VALOR IS NOT NULL AND  FLAG_PERIODO IN ('LY','CY') 

GROUP BY
RESPID,
QUESTIONARIO,
FLAG_PERIODO,
BANDEIRA,
DATA_ENTREVISTA,
DATE_ID,
ANO,
NOME_SEGMENTO,
SEXO,
IDADE,
UF,
DEPTO,
SETOR,
DS_TIPO_ENTREGA,
METODOLOGIA
"""

###############################################################################################################################

DETRATORES="""
SELECT
RESPID,
QUESTIONARIO,
FLAG_PERIODO,
BANDEIRA,
DATA_ENTREVISTA,
DATE_ID,
ANO,
NOME_SEGMENTO,
SEXO,
IDADE,
UF,
DEPTO,
SETOR,
DS_TIPO_ENTREGA,
METODOLOGIA,
COUNT(VALOR) AS QTD_DETRATORES

FROM vv_base_dash

WHERE DIMENSAO = 'NPS' AND VALOR IN (0,1,2,3,4,5,6) AND  FLAG_PERIODO IN ('LY','CY')

GROUP BY
RESPID,
QUESTIONARIO,
FLAG_PERIODO,
BANDEIRA,
DATA_ENTREVISTA,
DATE_ID,
ANO,
NOME_SEGMENTO,
SEXO,
IDADE,
UF,
DEPTO,
SETOR,
DS_TIPO_ENTREGA,
METODOLOGIA
"""

##############################################################################################################################

NEUTROS="""
SELECT
RESPID,
QUESTIONARIO,
FLAG_PERIODO,
BANDEIRA,
DATA_ENTREVISTA,
DATE_ID,
ANO,
NOME_SEGMENTO,
SEXO,
IDADE,
UF,
DEPTO,
SETOR,
DS_TIPO_ENTREGA,
METODOLOGIA,
COUNT(VALOR) AS QTD_NEUTROS

FROM vv_base_dash

WHERE DIMENSAO = 'NPS' AND VALOR IN (7,8) AND  FLAG_PERIODO IN ('LY','CY')

GROUP BY
RESPID,
QUESTIONARIO,
FLAG_PERIODO,
BANDEIRA,
DATA_ENTREVISTA,
DATE_ID,
ANO,
NOME_SEGMENTO,
SEXO,
IDADE,
UF,
DEPTO,
SETOR,
DS_TIPO_ENTREGA,
METODOLOGIA
"""

##############################################################################################################################

PROMOTORES="""
SELECT

RESPID,
QUESTIONARIO,
FLAG_PERIODO,
BANDEIRA,
DATA_ENTREVISTA,
DATE_ID,
ANO,
NOME_SEGMENTO,
SEXO,
IDADE,
UF,
DEPTO,
SETOR,
DS_TIPO_ENTREGA,
METODOLOGIA,
COUNT(VALOR) AS QTD_PROMOTORES

FROM vv_base_dash

WHERE DIMENSAO = 'NPS' AND VALOR IN (9,10) AND  FLAG_PERIODO IN ('LY','CY')

GROUP BY

RESPID,
QUESTIONARIO,
FLAG_PERIODO,
BANDEIRA,
DATA_ENTREVISTA,
DATE_ID,
ANO,
NOME_SEGMENTO,
SEXO,
IDADE,
UF,
DEPTO,
SETOR,
DS_TIPO_ENTREGA,
METODOLOGIA
"""
###############################################################################################################################

COUNT_TOTAL = pd.read_sql(COUNT_TOTAL, engine_OUT)
DETRATORES = pd.read_sql(DETRATORES, engine_OUT)
NEUTROS = pd.read_sql(NEUTROS, engine_OUT)
PROMOTORES = pd.read_sql(PROMOTORES, engine_OUT)
# NPS_LY = pd.read_sql(NPS_LY, engine_OUT)
# NPS_LY.columns = map(unicode.lower, NPS_LY.columns)

In [15]:
COUNT_TOTAL

,respid,questionario,flag_periodo,bandeira,data_entrevista,date_id,ano,nome_segmento,sexo,idade,uf,depto,setor,ds_tipo_entrega,metodologia,total
0,64579,B2C,CY,EX,/,0,0.0,B2C,0,0,0,0,0,0,0,1
1,63042,B2C,CY,EX,/,0,0.0,B2C,0,0,0,0,0,0,0,1
2,44446,B2C,CY,CB,/,0,0.0,B2C,0,0,0,0,0,0,0,1
3,59784,B2C,CY,CB,/,0,0.0,B2C,0,0,0,0,0,0,0,1
4,52965,B2C,CY,EX,/,0,0.0,B2C,0,0,0,0,0,0,0,1
5,61970,B2C,CY,PF,/,0,0.0,B2C,0,0,0,0,0,0,0,1
6,229464,B2C,CY,EX,02/18,28/02/2018,2018.0,B2C,0,0,0,Eletroportateis,Cafeteira Filtro,Normal,0,1
7,177887,B2C,CY,EX,02/18,08/02/2018,2018.0,B2C,0,0,0,Eletroportateis,Fritadeiras,Normal,0,1
8,195234,B2C,CY,CB,02/18,15/02/2018,2018.0,B2C,0,0,0,Beleza e Saude,Pranchas (Chapinhas),Normal,0,1
9,199514,B2C,CY,EX,02/18,16/02/2018,2018.0,B2C,0,0,0,Utilidades Domesticas,Panelas,Normal,0,1


In [16]:
# NPS=[COUNT_TOTAL,PROMOTORES,DETRATORES]
# NPS = pd.concat(NPS)
NPS = pd.merge(PROMOTORES, DETRATORES, how= 'outer', on=['respid','date_id','data_entrevista','questionario','flag_periodo','bandeira','ano','uf','nome_segmento','depto','setor','sexo','idade','ds_tipo_entrega','metodologia'])
NPS = pd.merge(NPS, NEUTROS, how= 'outer', on=['respid','date_id','data_entrevista','questionario','flag_periodo','bandeira','ano','uf','nome_segmento','depto','setor','sexo','idade','ds_tipo_entrega','metodologia'])
NPS = pd.merge(NPS,COUNT_TOTAL, how= 'outer', on=['respid','date_id','data_entrevista','questionario','flag_periodo','bandeira','ano','uf','nome_segmento','depto','setor','sexo','idade','ds_tipo_entrega','metodologia'])

###REPLACING NaN FOR 0 ###
NPS = NPS.fillna(0)

###PERCENTUAL PROMOTORES###
NPS['percent_prom'] = (NPS['qtd_promotores']/NPS['total'])*100

###PERCENTUAL DETRATORES###
NPS['percent_detrat']= (NPS['qtd_detratores']/NPS['total'])*100


###PERCENTUAL NEUTROS###
NPS['percent_neutros']= (NPS['qtd_neutros']/NPS['total'])*100


###CONVERTING TO FLOAT###
NPS['percent_prom'] = NPS['percent_prom'].astype(float)
NPS['percent_detrat'] = NPS['percent_detrat'].astype(float)
NPS['percent_neutros'] = NPS['percent_neutros'].astype(float)
NPS['qtd_promotores'] = NPS['qtd_promotores'].astype(float)
NPS['qtd_detratores'] = NPS['qtd_detratores'].astype(float)
NPS['qtd_neutros'] = NPS['qtd_neutros'].astype(float)
NPS['total'] = NPS['total'].astype(float)


###CONVERTING TO STR###
NPS['flag_periodo'] = NPS['flag_periodo'].astype(str)
NPS['bandeira'] = NPS['bandeira'].astype(str)
NPS['data_entrevista'] = NPS['data_entrevista'].astype(str)
NPS['date_id'] = NPS['date_id'].astype(str)
NPS['nome_segmento'] = NPS['nome_segmento'].astype(str)
NPS['uf'] = NPS['uf'].astype(str)
NPS['ano'] = NPS['ano'].astype(str)
NPS['depto'] = NPS['depto'].astype(str)
NPS['setor'] = NPS['setor'].astype(str)
NPS['ds_tipo_entrega'] = NPS['ds_tipo_entrega'].astype(str)
NPS['questionario'] = NPS['questionario'].astype(str)
NPS['sexo'] = NPS['sexo'].astype(str)
NPS['idade'] = NPS['idade'].astype(str)
NPS['metodologia'] = NPS['metodologia'].astype(str)
# NPS['sexo'] = NPS['sexo'].astype(str)
# NPS['idade'] = NPS['idade'].astype(str)
# NPS['tipo_entrega'] = NPS['tipo_entrega'].astype(str)

###VALOR NPS###
NPS['nps'] = NPS['percent_prom']-NPS['percent_detrat']


###APPENDING NPS FROM THE LY###
# NPS = NPS.append(NPS_LY)


for column_name, column in NPS.transpose().iterrows():
    print column_name
    print(NPS[column_name])
    NPS[column_name] = NPS[column_name].astype('str')

NPS.to_sql('vv_nps',engine_OUT,schema=None,if_exists='replace', chunksize=500, index=False,dtype={col_name: NVARCHAR(300) for col_name in NPS})

print('DONE NPS')
print('DONE PROCESSO DASH VIA VAREJO')

respid
0          64579
1          44446
2          59784
3          52965
4          61970
5         229464
6         199514
7         121867
8         120936
9         145945
10        131991
11        180252
12         41637
13         61061
14         59589
15         24014
16         43180
17        227359
18         41796
19        153400
20        202734
21        179417
22        191004
23        206688
24         41252
25        187874
26        191848
27        182959
28        173710
29        160955
           ...  
490220         0
490221         0
490222         0
490223         0
490224         0
490225         0
490226         0
490227         0
490228         0
490229         0
490230         0
490231         0
490232         0
490233         0
490234         0
490235         0
490236         0
490237         0
490238         0
490239         0
490240         0
490241         0
490242         0
490243         0
490244         0
490245         0
490246         0
490247 

sexo
0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         M
8         F
9         M
10        F
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        F
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        M
         ..
490220    F
490221    F
490222    M
490223    M
490224    F
490225    F
490226    F
490227    M
490228    M
490229    F
490230    F
490231    M
490232    F
490233    M
490234    M
490235    M
490236    M
490237    M
490238    M
490239    F
490240    M
490241    M
490242    F
490243    M
490244    M
490245    M
490246    F
490247    F
490248    M
490249    F
Name: sexo, Length: 490250, dtype: object
idade
0               0
1               0
2               0
3               0
4               0
5               0
6               0
7         18 - 24
8         18 - 24
9         36 - 45
10        25 - 35
11              0

metodologia
0                      0
1                      0
2                      0
3                      0
4                      0
5                      0
6                      0
7                      0
8                      0
9                      0
10                     0
11                     0
12                     0
13                     0
14                     0
15                     0
16                     0
17                     0
18                     0
19                     0
20                     0
21                     0
22                     0
23                     0
24                     0
25                     0
26                     0
27                     0
28                     0
29                     0
               ...      
490220                 0
490221                 0
490222    Data Prometida
490223                 0
490224      Data Entrega
490225                 0
490226                 0
490227                 0
490228    Dat

nps
0         100.0
1         100.0
2         100.0
3         100.0
4         100.0
5         100.0
6         100.0
7         100.0
8         100.0
9         100.0
10        100.0
11        100.0
12        100.0
13        100.0
14        100.0
15        100.0
16        100.0
17        100.0
18        100.0
19        100.0
20        100.0
21        100.0
22        100.0
23        100.0
24        100.0
25        100.0
26        100.0
27        100.0
28        100.0
29        100.0
          ...  
490220      0.0
490221      0.0
490222      0.0
490223      0.0
490224      0.0
490225      0.0
490226      0.0
490227      0.0
490228      0.0
490229      0.0
490230      0.0
490231      0.0
490232      0.0
490233      0.0
490234      0.0
490235      0.0
490236      0.0
490237      0.0
490238      0.0
490239      0.0
490240      0.0
490241      0.0
490242      0.0
490243      0.0
490244      0.0
490245      0.0
490246      0.0
490247      0.0
490248      0.0
490249      0.0
Name: nps, Length: 4

In [17]:
NPS

,respid,questionario,flag_periodo,bandeira,data_entrevista,date_id,ano,nome_segmento,sexo,idade,...,ds_tipo_entrega,metodologia,qtd_promotores,qtd_detratores,qtd_neutros,total,percent_prom,percent_detrat,percent_neutros,nps
0,64579,B2C,CY,EX,/,0,0.0,B2C,0,0,...,0,0,1.0,0.0,0.0,1.0,100.0,0.0,0.0,100.0
1,44446,B2C,CY,CB,/,0,0.0,B2C,0,0,...,0,0,1.0,0.0,0.0,1.0,100.0,0.0,0.0,100.0
2,59784,B2C,CY,CB,/,0,0.0,B2C,0,0,...,0,0,1.0,0.0,0.0,1.0,100.0,0.0,0.0,100.0
3,52965,B2C,CY,EX,/,0,0.0,B2C,0,0,...,0,0,1.0,0.0,0.0,1.0,100.0,0.0,0.0,100.0
4,61970,B2C,CY,PF,/,0,0.0,B2C,0,0,...,0,0,1.0,0.0,0.0,1.0,100.0,0.0,0.0,100.0
5,229464,B2C,CY,EX,02/18,28/02/2018,2018.0,B2C,0,0,...,Normal,0,1.0,0.0,0.0,1.0,100.0,0.0,0.0,100.0
6,199514,B2C,CY,EX,02/18,16/02/2018,2018.0,B2C,0,0,...,Normal,0,1.0,0.0,0.0,1.0,100.0,0.0,0.0,100.0
7,121867,B2C,CY,PF,01/18,19/01/2018,2018.0,B2C,M,18 - 24,...,Normal,0,1.0,0.0,0.0,1.0,100.0,0.0,0.0,100.0
8,120936,B2C,CY,CB,01/18,19/01/2018,2018.0,B2C,F,18 - 24,...,Normal,0,1.0,0.0,0.0,1.0,100.0,0.0,0.0,100.0
9,145945,B2C,CY,PF,01/18,28/01/2018,2018.0,B2C,M,36 - 45,...,Normal,0,1.0,0.0,0.0,1.0,100.0,0.0,0.0,100.0


<b><h1>EXPORTING BASES</b></h1>

In [90]:
VV_BASE_JAN = """SELECT * FROM vv_base_dash_new WHERE DATA_ENTREVISTA = '01/18'"""
VV_BASE_JAN = pd.read_sql(VV_BASE_JAN, engine_OUT)
VV_BASE_JAN
VV_BASE_JAN.to_csv('/nfs/science/casino_brazil/luizsalv/Via_Varejo/ouput/vv_base_jan.csv',encoding = 'UTF-8',index=False,header=True,mode='w')
# NPS.to_csv('/nfs/science/casino_brazil/luizsalv/Via_Varejo/ouput/vv_base_NPS.csv',encoding = 'UTF-8',index=False,header=True,mode='w')

In [103]:
VV_BASE_FEV = """SELECT COUNT(*) FROM vv_base_dash_new WHERE DATA_ENTREVISTA = '02/18'"""
VV_BASE_FEV = pd.read_sql(VV_BASE_FEV, engine_OUT)
VV_BASE_FEV
# VV_BASE_FEV.to_csv('/nfs/science/casino_brazil/luizsalv/Via_Varejo/ouput/vv_base_fev.csv',encoding = 'UTF-8',index=False,header=True,mode='w')

,COUNT(*)
0,887712


In [13]:
# df1 = df.replace(np.nan, '', regex=True)

vv_base_dash="""CREATE TABLE vv_base_dash AS SELECT 
CASE WHEN ANO= 'nan' THEN '0' ELSE ANO END AS ANO,
ATRIBUTO,
BANDEIRA,
BANDEIRA_NUM_PEDIDO,
CASE WHEN DATA_ENTREVISTA IN ('/','nan') THEN '0' ELSE DATA_ENTREVISTA END AS DATA_ENTREVISTA,
CASE WHEN DATE_ID IN ('/','nan') THEN '0' ELSE DATE_ID END AS DATE_ID,
DEPTO,
DIMENSAO,
DS_TIPO_ENTREGA,
FLAG_PERIODO,
IDADE,
ID_BANDEIRA,
CASE WHEN ID_COMPRA = 'nan' THEN '0' ELSE ID_COMPRA END AS ID_COMPRA,
ID_ENTREGA,
ID_QUESTIONARIO,
ID_SEGMENTO,
ISC,
CASE WHEN LOJISTA = 'nan' THEN '0' ELSE LOJISTA END AS LOJISTA,
CASE WHEN METODOLOGIA = 'nan' THEN '0' ELSE METODOLOGIA END AS METODOLOGIA,
NOME_SEGMENTO,
CASE WHEN NR_LOJISTA = 'nan' then '0' else NR_LOJISTA END AS NR_LOJISTA,
QUESTIONARIO,
RESPID,
SETOR,
SEXO,
TIPO_ENTREGA,
UF,
VALOR,
CASE WHEN "RANK" = 'nan' THEN '0' ELSE "RANK" END AS RANK,
SEGMENTO

FROM AN_RT_WS36.VV_BASE_DASH_NEW"""
# vv_base_dash = vv_base_dash.fillna(0)
defs.dropTable('vv_base_dash', engine_OUT)
defs.sqlProcess(vv_base_dash, 'vv_base_dash', engine_OUT)
print('done query DASH')

DatabaseError: (cx_Oracle.DatabaseError) ORA-00904: "/": invalid identifier [SQL: 'CREATE TABLE vv_base_dash AS SELECT \nCASE WHEN ANO= \'nan\' THEN \'0\' ELSE ANO END AS ANO,\nATRIBUTO,\nBANDEIRA,\nBANDEIRA_NUM_PEDIDO,\nCASE WHEN DATA_ENTREVISTA IN ("/",\'nan\') THEN \'0\' ELSE DATA_ENTREVISTA END AS DATA_ENTREVISTA,\nCASE WHEN DATE_ID IN ("/",\'nan\') THEN \'0\' ELSE DATE_ID END AS DATE_ID,\nDEPTO,\nDIMENSAO,\nDS_TIPO_ENTREGA,\nFLAG_PERIODO,\nIDADE,\nID_BANDEIRA,\nCASE WHEN ID_COMPRA = \'nan\' THEN \'0\' ELSE ID_COMPRA END AS ID_COMPRA,\nID_ENTREGA,\nID_QUESTIONARIO,\nID_SEGMENTO,\nISC,\nCASE WHEN LOJISTA = \'nan\' THEN \'0\' ELSE LOJISTA END AS LOJISTA,\nCASE WHEN METODOLOGIA = \'nan\' THEN \'0\' ELSE METODOLOGIA END AS METODOLOGIA,\nNOME_SEGMENTO,\nCASE WHEN NR_LOJISTA = \'nan\' then \'0\' else NR_LOJISTA END AS NR_LOJISTA,\nQUESTIONARIO,\nRESPID,\nSETOR,\nSEXO,\nTIPO_ENTREGA,\nUF,\nVALOR,\nCASE WHEN "RANK" = \'nan\' THEN \'0\' ELSE "RANK" END AS RANK,\nSEGMENTO\n\nFROM AN_RT_WS36.